# Introduction

The aim of this notebook is to provide an estimate of the probability of being dealt a 13-card hand that qualifies for Fantasyland in the Open Face Chinese Poker (OFCP). <br>

I used the Monte Carlo Simulation to approximate the result, in general the Monte Carlo simulation invloves :

> 1.) Repeatedly select random data points
> > Generating 13-card hands at random.

> 2.) Perform a deterministic computation
> > This is the hard part : I defined a function `DECONSTRUCTOR` that breaks down each generated 13-card hand into simpler discrete `registered(combinatorial) units`. Later these 'registered units' are grouped together in `grouper3` to find a mutually exclusive grouping that either satisfies the Fantasyland condition or not.

> 3.) Combine the results
> > Track the number of times each 13-card hand satisfies Fantasyland or not.

## Static variables
Creating the the deck, premium_cards, the two rank orderings, and pleb_cards.
> Any collection of cards will be stored as a list of tuples, where `tup[0]` is rank, and `tup[1]` is suit.

In [1]:
import random
from collections import Counter

# Define the ranks and suits of a deck of cards
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
ranks_wheel = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'] # In
suits = ['Spades', 'Hearts', 'Diamonds', 'Clubs']

# Create a deck of cards
deck = [(rank, suit) for suit in suits for rank in ranks]

premium_ranks = ['Q', 'K', 'A']
premium_cards = [(rank, suit) for suit in suits for rank in premium_ranks]
pleb_cards = [(rank, suit) for suit in suits for rank in (set(ranks)-set(premium_ranks))]

## Edgecases - Static

In [2]:
import pandas as pd 
pd.set_option('display.max_colwidth', None) 
df = pd.read_excel(r'./Data/Edgecases.xlsx')
display(df)

,Name,Hand,Remarks
0,01_threequeens_edgecase_flush_str,"[('Q', 'Spades'), ('Q', 'Diamonds'), ('2', 'Diamonds'), ('4', 'Diamonds'), ('6', 'Diamonds'), ('8', 'Diamonds'), ('Q', 'Hearts'), ('K', 'Spades'), ('A', 'Clubs'), ('3', 'Clubs'), ('6', 'Hearts'), ('9', 'Spades'), ('J', 'Hearts')]",Contains 3 Queens where the the Queen of Diamonds is part of a 5 card flush. No fantasyland
1,02_threequeens_edgecase_straight_str,"[('8', 'Spades'), ('9', 'Diamonds'), ('10', 'Clubs'), ('J', 'Hearts'), ('Q', 'Hearts'), ('Q', 'Clubs'), ('Q', 'Diamonds'), ('10', 'Hearts'), ('A', 'Spades'), ('2', 'Clubs'), ('2', 'Hearts'), ('3', 'Diamonds'), ('5', 'Clubs')]","Contains 3 Queens where the Queen rank is part of a 8,9,10,J,Q (Q-high) straight.No Fantasyland"
2,03_dragon_hardcode_edgecase,"[('A', 'Spades'), ('2', 'Hearts'), ('3', 'Spades'), ('4', 'Diamonds'), ('5', 'Clubs'), ('6', 'Diamonds'), ('7', 'Hearts'), ('8', 'Spades'), ('9', 'Clubs'), ('10', 'Clubs'), ('Q', 'Hearts'), ('K', 'Spades'), ('J', 'Hearts')]",One card from each rank. No Fantasyland
3,04_tripswith2straights_edgecase,"[('10', 'Diamonds'), ('2', 'Clubs'),('3', 'Hearts'),('4', 'Diamonds'),('5', 'Hearts'),('6', 'Hearts'),('8', 'Spades'),('9', 'Diamonds'),('A', 'Clubs'),('A', 'Hearts'),('J', 'Spades'),('A', 'Diamonds'),('Q', 'Diamonds')]","Ace triple and 2,3,4,5,6 and 8,9,10,J,Q straights.Fantasyland"
4,05_twoQuads_str,"[('K', 'Hearts'),('K', 'Clubs'),('9', 'Hearts'),('J', 'Hearts'),('8', 'Diamonds'),('7', 'Spades'),('K', 'Diamonds'),('7', 'Hearts'),('10', 'Clubs'),('K', 'Spades'),('Q', 'Hearts'),('7', 'Diamonds'),('7', 'Clubs')]",King and 7 both are Quads. Fantasyland.
5,06_2quads and a straight flush,"[('10', 'Hearts'), ('10', 'Clubs'), ('10', 'Spades'), ('10', 'Diamonds'), ('A', 'Spades'), ('2', 'Spades'), ('3', 'Spades'), ('4', 'Spades'), ('5', 'Spades'), ('K', 'Hearts'), ('K', 'Spades'), ('K', 'Clubs'), ('K', 'Diamonds')]","triggers fantast for double kings, and quad 10, perhaps develop for all 13 synergy?"
6,07_interaction_flush_got_2_straights_edgecase_str,"[('4', 'Diamonds'), ('5', 'Hearts'), ('5', 'Diamonds'), ('6', 'Spades'), ('7', 'Spades'), ('7', 'Hearts'), ('7', 'Clubs'), ('7', 'Diamonds'), ('8', 'Spades'), ('8', 'Diamonds'), ('9', 'Diamonds'), ('10', 'Diamonds'), ('J', 'Diamonds'),]\n","Contains a 2 straight, and a flush. Classic Case_1 Both Flush and One Long straight from 4,5,6,7,8,9,10,J inclusive bounds. 7 & 8 are duplicates."
7,08_interaction_flush_got_2_straight_1_good_straight,"[('4', 'Diamonds'), ('5', 'Hearts'), ('5', 'Diamonds'), ('6', 'Spades'), ('7', 'Spades'), ('7', 'Hearts'), ('7', 'Clubs'), ('7', 'Diamonds'), ('8', 'Spades'), ('8', 'Diamonds'), ('9', 'Hearts'), ('10', 'Diamonds'), ('J', 'Diamonds')]\n","('9', 'Diamonds') is subed for ('9', 'Hearts'). This time there are no loaned_flushed_cards"
8,09_interaction_flush_no2straights_edgecase_str,"[('4', 'Diamonds'), ('5', 'Hearts'), ('5', 'Diamonds'), ('6', 'Spades'), ('6', 'Diamonds'), ('7', 'Spades'), ('7', 'Hearts'), ('7', 'Clubs'), ('7', 'Diamonds'), ('8', 'Spades'), ('9', 'Hearts'), ('10', 'Diamonds'), ('J', 'Diamonds'),]\n","Continueing from prev TC, ('8', 'Diamonds') is subed for ('6', 'Diamonds'). 2 overlapping straights no longer found because central cards 7 and 8 are not duplicate. This actually doesn't factor because it is case_1, expected result bad TC."
9,10_2_long_straight_involving_ACES,"[('2', 'Spades', ), ('3', 'Diamonds', ), ('4', 'Hearts', ), ('5', 'Hearts', ), ('5', 'Clubs', ), ('6', 'Spades', ), ('9', 'Clubs', ), ('10', 'Hearts', ), ('J', 'Diamonds', ), ('J', 'Clubs', ), ('Q', 'Clubs', ), ('K', 'Spades', ), ('A', 'Spades', )]","A couple of 5's, Ace-Low, and Ace-High straights. No Fantasy."


In [3]:
from itertools import cycle
import ast
cycle_suits = cycle(suits)
non_flush_edgecase_template = [ast.literal_eval(f"(\'\',\'{next(cycle_suits)}\')") for _ in range(13)]
print(non_flush_edgecase_template)

[('', 'Spades'), ('', 'Hearts'), ('', 'Diamonds'), ('', 'Clubs'), ('', 'Spades'), ('', 'Hearts'), ('', 'Diamonds'), ('', 'Clubs'), ('', 'Spades'), ('', 'Hearts'), ('', 'Diamonds'), ('', 'Clubs'), ('', 'Spades')]


In [4]:
from pprint import pprint
# Convert pandas dataframe into dict with Name as keys
my_hands = {}
for index, row_dict in df.to_dict(orient='index').items():
    my_hands.update({ row_dict['Name'] : {'Hand' : row_dict['Hand'],
                                          'Remarks': row_dict['Remarks']} })
pprint(my_hands)

{'01_threequeens_edgecase_flush_str': {'Hand': "[('Q', 'Spades'), ('Q', "
                                               "'Diamonds'), ('2', "
                                               "'Diamonds'), ('4', "
                                               "'Diamonds'), ('6', "
                                               "'Diamonds'), ('8', "
                                               "'Diamonds'), ('Q', 'Hearts'), "
                                               "('K', 'Spades'), ('A', "
                                               "'Clubs'), ('3', 'Clubs'), "
                                               "('6', 'Hearts'), ('9', "
                                               "'Spades'), ('J', 'Hearts')]",
                                       'Remarks': 'Contains 3 Queens where the '
                                                  'the Queen of Diamonds is '
                                                  'part of a 5 card flush. No '
                               

In [5]:
import re
print(f"This is the last static edgecase in the excel file : {df.at[len(df)-1, 'Name']}")

number = re.search(pattern=r'^(\d*)', string=df.at[len(df)-1, 'Name'])
new_num = int(number.group(1)) + 1

This is the last static edgecase in the excel file : 15_Case2_BigOverlap


## Add random hands
> Add hands

In [6]:
random_dragon_edgecase = str([(rank, random.choice(suits)) for rank in ranks])
# Appending immutably 
premium_cards_plus1random_edgecase = str([*premium_cards, ((random.choice(pleb_cards)))])
# Draw the 4 Queens and 9 random plebeian cards
fourqueens = [('Q', 'Spades'), ('Q', 'Hearts'), ('Q', 'Clubs'), ('Q', 'Diamonds')]
fourqueens_and_plebs_random_edgecase = str([*fourqueens, *((random.sample(pleb_cards, 9)))])

my_hands.update({f'{new_num:02d}_random_dragon_edgecase': {'Hand' : random_dragon_edgecase, 'Remarks': None},   
                     f'{new_num+1:02d}_premium_cards_plus1random_edgecase': {'Hand' : premium_cards_plus1random_edgecase, 'Remarks': None},
                    f'{new_num+2:02d}_fourqueens_and_plebs_random_edgecase': {'Hand' : fourqueens_and_plebs_random_edgecase, 'Remarks': None}
                })

In [7]:
available_hands = list(my_hands.keys())
pprint(available_hands)

['01_threequeens_edgecase_flush_str',
 '02_threequeens_edgecase_straight_str',
 '03_dragon_hardcode_edgecase',
 '04_tripswith2straights_edgecase',
 '05_twoQuads_str',
 '06_2quads and a straight flush',
 '07_interaction_flush_got_2_straights_edgecase_str',
 '08_interaction_flush_got_2_straight_1_good_straight',
 '09_interaction_flush_no2straights_edgecase_str',
 '10_2_long_straight_involving_ACES',
 '11_3_monsters',
 '12_2_flushes_and_2_long_straights',
 '13_Case2_one_straight_has_no_dupilcate_ranks',
 '14_Case2_all_substraights_have_duplicates',
 '15_Case2_overlap',
 '15_Case2_BigOverlap',
 '16_random_dragon_edgecase',
 '17_premium_cards_plus1random_edgecase',
 '18_fourqueens_and_plebs_random_edgecase']


In [8]:
my_hands['04_tripswith2straights_edgecase']['Hand']

"[('10', 'Diamonds'), ('2', 'Clubs'),('3', 'Hearts'),('4', 'Diamonds'),('5', 'Hearts'),('6', 'Hearts'),('8', 'Spades'),('9', 'Diamonds'),('A', 'Clubs'),('A', 'Hearts'),('J', 'Spades'),('A', 'Diamonds'),('Q', 'Diamonds')]"

In [9]:
# Extract prefix numbers and convert to integers
high_integer = max([int(re.search(r'^(\d+)', item).group(1)) for item in available_hands])
high_integer

18

## Generate edgecase

In [10]:
# Four of a kind
def generate_four_of_a_kind_edgecase(is_random=True, is_exclude_other_prems=False):
    """
    Explicitly outputs a one 4-of-a-kind, of a rank of choice or random. Other 9 cards are random,
    """
    if is_random:
        # Returns a single element list
        random_rank = random.sample(ranks, 1)
        # Convert list to str with .join
        random_rank = ''.join([str(one_rank) for one_rank in random_rank])
    else:
        random_rank = input('Input a rank')

    # Get the 4-of-a-kind
    fourOfaKind = [(random_rank,  suit) for suit in suits]

    if is_exclude_other_prems:
        # Pack the other pleb cards
        other_9 = [(rank, suit) for rank, suit in random.sample(pleb_cards, 9) if rank != random_rank]
    else:
        other_9 = [(rank, suit) for rank, suit in random.sample(deck, 9) if rank != random_rank ]
    # Pack the other 9 cards in
    fourOfaKind = ([*fourOfaKind,
                    *other_9
                   ])
    # fourOfaKind_str = str(fourOfaKind)
    return str(fourOfaKind)


def generate_curious_edgecase():
    """
    10 J Q K A
           K A 2 3 4 5     7 8
    
    """
    curious_case = ['10', 'J', 'Q', '2', '3', '4', '5', '7', '8']
    # Get two random kings and two random aces 
    two_kings = random.sample([card for card in premium_cards if card[0] == 'K'], 2)
    two_aces = random.sample([card for card in premium_cards if card[0] == 'A'], 2)
    curious_case = ([*[(rank, random.choice(suits)) for rank in curious_case], *two_kings, *two_aces])
    # curious_case_str = str(curious_case)
    return str(curious_case)

In [11]:
generate_4_of_a_kind_edgecase = generate_four_of_a_kind_edgecase(is_random=True, is_exclude_other_prems=False)
generate_curious_edgecase_str = generate_curious_edgecase()

In [12]:
generate_curious_edgecase_str

"[('10', 'Diamonds'), ('J', 'Spades'), ('Q', 'Spades'), ('2', 'Hearts'), ('3', 'Spades'), ('4', 'Clubs'), ('5', 'Diamonds'), ('7', 'Hearts'), ('8', 'Clubs'), ('K', 'Diamonds'), ('K', 'Hearts'), ('A', 'Hearts'), ('A', 'Diamonds')]"

## Draw cards with IDs

In [13]:
pprint(my_hands)

{'01_threequeens_edgecase_flush_str': {'Hand': "[('Q', 'Spades'), ('Q', "
                                               "'Diamonds'), ('2', "
                                               "'Diamonds'), ('4', "
                                               "'Diamonds'), ('6', "
                                               "'Diamonds'), ('8', "
                                               "'Diamonds'), ('Q', 'Hearts'), "
                                               "('K', 'Spades'), ('A', "
                                               "'Clubs'), ('3', 'Clubs'), "
                                               "('6', 'Hearts'), ('9', "
                                               "'Spades'), ('J', 'Hearts')]",
                                       'Remarks': 'Contains 3 Queens where the '
                                                  'the Queen of Diamonds is '
                                                  'part of a 5 card flush. No '
                               

In [14]:
import ast
## Define function to draw cards at random, with IDs
def draw_cards_with_IDs(is_random, edgecase_str=my_hands['01_threequeens_edgecase_flush_str']):
    """
    Assign a ID_number to each card by extending to card_tuple -> (rank, suit, ID)
    """
    new_list = []
    if is_random == True:
        for i, e_tuple in enumerate(random.sample(deck, 13), 1):
            element = (*e_tuple, f'ID_{i}')
            new_list.append(element)
    elif is_random == False:
        # Process the raw string into a Python object.
        edgecase = ast.literal_eval(edgecase_str)
        for i, e_tuple in enumerate(edgecase, 1):
            element = (*e_tuple, f'ID_{i}')
            new_list.append(element)
    return new_list

# Invoke function
draw_cards_with_IDs(is_random=False, edgecase_str=generate_curious_edgecase_str)

[('10', 'Diamonds', 'ID_1'),
 ('J', 'Spades', 'ID_2'),
 ('Q', 'Spades', 'ID_3'),
 ('2', 'Hearts', 'ID_4'),
 ('3', 'Spades', 'ID_5'),
 ('4', 'Clubs', 'ID_6'),
 ('5', 'Diamonds', 'ID_7'),
 ('7', 'Hearts', 'ID_8'),
 ('8', 'Clubs', 'ID_9'),
 ('K', 'Diamonds', 'ID_10'),
 ('K', 'Hearts', 'ID_11'),
 ('A', 'Hearts', 'ID_12'),
 ('A', 'Diamonds', 'ID_13')]

## Auxillary functions

In [15]:
from collections import defaultdict

def new_rank_function(lst_sample, custom_order, list_of_straight_ranks=list(), is_print=True):
    '''
    Given a 13 card sample, and a given straight-rank-order ie ranks (A-K) or (2-A), ie custom_order, 
    Get the ranks associated with the straight

    list_of_ranks_Ahigh=new_rank_function(d['sample'],
                                          custom_order=ranks,
                                          list_of_straight_ranks=list())

    # Then feed the variable as the argument's value for the argument : list_of_straight_ranks
    all_straight_ranks = new_rank_function(d['sample'],
                                           custom_order=ranks_wheel,
                                           list_of_straight_ranks=list_of_ranks_Ahigh)
    '''


    # 1️⃣  Group all cards by rank.
    #     Example: {'5': [5♦, 5♥], '6': [6♠], ...}
    rank_to_cards = defaultdict(list)
    for card in lst_sample:
        rank_to_cards[card[0]].append(card)

    # 2️⃣  Create a lookup rank -> index based on custom_order (e.g. '2' = 0, '3' = 1, etc.)
    rank_to_index = {rank: i for i, rank in enumerate(custom_order)}

    # 3️⃣  Collect unique ranks in this hand, and sort them by rank order.
    unique_ranks = sorted(rank_to_cards.keys(), key=lambda r: rank_to_index[r])
    i=0
    while i < len(unique_ranks):
        # 4️⃣  Start building a run of consecutive unique ranks.
        straight_ranks = [unique_ranks[i]]
        if is_print:
            print(f'Starting a new straight at index {i} with the element : " \n{straight_ranks}')
        j = i + 1
        while j < len(unique_ranks):
            # Check if the next rank is consecutive.
            if rank_to_index[unique_ranks[j]] == rank_to_index[unique_ranks[j - 1]] + 1:
                if is_print:
                    print(f'Appending the rank {unique_ranks[j]}')
                straight_ranks.append(unique_ranks[j])
                j += 1
            else:
                # Sequence break -> stop extending this run.
                break
        # 6️⃣  Move the outer loop pointer to the end of this run.
        i = j
    
        # 5️⃣  If the run is long enough,
        if len(straight_ranks) >= 5 and straight_ranks not in list_of_straight_ranks :
            print(f'Here is the long straight {straight_ranks}')
            list_of_straight_ranks.append(straight_ranks)

    return list_of_straight_ranks

def remove_subsets(list_of_lists):
    result = []
    seen = set()

    for cur_sublist in list_of_lists:
        # normalize for permutation detection
        normalized = tuple(sorted(cur_sublist))

        # skip if we’ve already seen this combination
        if normalized in seen:
            continue

        # check if cur_sublist is a proper subset of any other
        if not any(set(cur_sublist).issubset(set(other)) and set(cur_sublist) != set(other)
                   for other in list_of_lists):
            result.append(cur_sublist)
            seen.add(normalized)

    return result

lists = [[1,2], [1,2,3], [4], [4,5], [1]]
filtered = remove_subsets(lists)
print(filtered)

def get_central_slice(lst):
    """
    If a 5+ run or straight is found, this function will return the central_ranks
    This is an auxillary function, if all ranks in the central_ranks are duplicated, then we can be sure to construct 2 straights from the 5+ run
    """
    central_slice_len = 10 - len(lst)
    center = len(lst) // 2 
    half = central_slice_len // 2
    print(f'\nThis is the center index : {center} which maps to element [{lst[center]}] and half value of {half}')

    if central_slice_len % 2 == 0:
        start = center - half
        print(f'This is the start index : {start} which maps to this element : {lst[start]}')
        end = center + half
        print(f'This is the end index : {end} for even-length list/csl which maps to {lst[end]}. Exclusive')
    else:
        start = center - half
        print(f'This is the start index : {start} which maps to this element : {lst[start]}')
        end = center + half + 1 
        print(f'This is the end index : {end} for odd-length list/csl which maps to  {lst[end-1]}. Inclusive')
    
 
    return lst[:start],lst[start:end], lst[end:]

print(get_central_slice(['a', 'b', 'c', 'd', 'e', 'f', 'g']))  
# len = 7 → central_len = 3 → output: ['c', 'd', 'e']

print(get_central_slice(['a', 'b', 'c', 'd', 'e', 'f']))  
# len = 6 → central_len = 4 → output: ['b', 'c', 'd', 'e']

print(get_central_slice(['a', 'b', 'c', 'd', 'e']))  

[[1, 2, 3], [4, 5]]

This is the center index : 3 which maps to element [d] and half value of 1
This is the start index : 2 which maps to this element : c
This is the end index : 5 for odd-length list/csl which maps to  e. Inclusive
(['a', 'b'], ['c', 'd', 'e'], ['f', 'g'])

This is the center index : 3 which maps to element [d] and half value of 2
This is the start index : 1 which maps to this element : b
This is the end index : 5 for even-length list/csl which maps to f. Exclusive
(['a'], ['b', 'c', 'd', 'e'], ['f'])

This is the center index : 2 which maps to element [c] and half value of 2
This is the start index : 0 which maps to this element : a
This is the end index : 5 for odd-length list/csl which maps to  e. Inclusive
([], ['a', 'b', 'c', 'd', 'e'], [])


In [16]:
remove_subsets([[1,3,2], [1,2]])

[[1, 3, 2]]

## Check functions
> It is easier to check if a given straight is a flush, then to check if a given flush is a straight.

In [17]:
def check_straight_is_straight_flush(straight:'list_of_tuples'):
    """
    Returns True if the input straight is a straight flush
    """
    straight_flush = []
    # Get a list of the suit combinations in the straight
    suits = [card[1] for card in straight]
    
    # Count frequency of each suit
    suit_counts = {suit: suits.count(suit) for suit in set(suits)}
    
    # Check if any suit appears 5 times (i.e., all cards are same suit)
    if any(count == 5 for count in suit_counts.values()):
        return True
    else:
        return False

def check_flush_is_straight_flush(flush:'list_of_tuples'):
    straight_flush = []
    low_straight = check_lexicographical_straight(lst=flush, custom_order=ranks,allow_substraights=True, return_type='card_tuple')
    high_straight = check_lexicographical_straight(lst=flush, custom_order=ranks_wheel,allow_substraights=True, return_type='card_tuple')
    if any([low_straight, high_straight]):
        return True
    else:
        return False

def lexicographical_straight_cutter(long_straight, connections):
    """
    Slice a long straight of card tuples into overlapping straights
    of length (connections + 1).
    """
    sub_straights = []
    for i in range(len(long_straight) - connections):
        sub = long_straight[i : i + connections + 1]
        sub_straights.append(sub)
    return sub_straights


def check_lexicographical_straight(lst, custom_order=ranks, atleast=4, allow_substraights=True, return_type='card_tuple'):
    """
    Finds disjoint locally longest straights from card tuples and optionally 
    extracts all fixed-length substraights.
    
    Parameters:
    - lst: list of tuples (rank, suit, id)
    - atleast: minimum connections (e.g., 4 means 5-card straight)
    - allow_substraights: whether to allow slicing into overlapping substraights
    - return_type: 'card_tuple' for full tuple, 'rank' for just the rank
    """

    # Dict of rank, index pairs - based on either ranks or rankw
    rank_to_index = {rank: i for i, rank in enumerate(custom_order)}
    # Sort based on rank order
    lst_sorted = sorted(lst, key=lambda x: rank_to_index[x[0]])

    result_straights = []
    current_straight = []

    # Loop through sorted cards..
    for i, card in enumerate(lst_sorted):
        # On the very first iteration, append the card to current_straight
        if not current_straight:
            current_straight.append(card)
        # Otherwise - we are now from second card onwards...
        else:
            # Assign the latest appended rank as prev_rank
            prev_rank = current_straight[-1][0]
            # Get the current_rank
            current_rank = card[0]

            # Check if in sequence
            if rank_to_index[current_rank] == rank_to_index[prev_rank] + 1:
                current_straight.append(card)
            # Enhancement - in the event of pairs - we didn't consider this egdecase before
            elif rank_to_index[current_rank] == rank_to_index[prev_rank]:
                pass
            # Break in sequence — commit current if long enough
            else: 
                # If current_straight is long enough...
                if len(current_straight) >= atleast + 1: # bool_expression for legit straight
                    result_straights.append(current_straight.copy()) # ... append it to result_straights
                # Redefine current_straight - preparing it for the next iteration
                current_straight = [card] # card here will become current_straight[-1][0] in the next iteration.

    # Final check - if we get to the last card without breaking sequence
    if len(current_straight) >= atleast + 1: # bool_expression for legit straight
        result_straights.append(current_straight)

    if allow_substraights:
        final_subs = []
        for straight in result_straights:
            subs = lexicographical_straight_cutter(straight, connections=atleast)
            final_subs.extend(subs)

        if final_subs:
            # print(f"Sub-straights of length {atleast + 1} found:")
            for s in final_subs:
                if return_type == 'rank':
                    print([c[0] for c in s])
                else:
                    print(s)
        else:
            pass
            # print(f"No sub-straights of length {atleast + 1} found.")
        return final_subs
    else:
        if result_straights:
            # print(f"Disjoint locally longest straights found:")
            for s in result_straights:
                if return_type == 'rank':
                    print(f"{len(s)}-card straight: {[c[0] for c in s]}")
                else:
                    pass
                    # print(f"{len(s)}-card straight: {s}")
        else:
            pass
            # print(f"No {atleast + 1}-card straight found.")
        return result_straights

In [18]:
h = draw_cards_with_IDs(is_random=False, edgecase_str=my_hands['07_interaction_flush_got_2_straights_edgecase_str']['Hand'])

In [19]:
pprint(h)

[('4', 'Diamonds', 'ID_1'),
 ('5', 'Hearts', 'ID_2'),
 ('5', 'Diamonds', 'ID_3'),
 ('6', 'Spades', 'ID_4'),
 ('7', 'Spades', 'ID_5'),
 ('7', 'Hearts', 'ID_6'),
 ('7', 'Clubs', 'ID_7'),
 ('7', 'Diamonds', 'ID_8'),
 ('8', 'Spades', 'ID_9'),
 ('8', 'Diamonds', 'ID_10'),
 ('9', 'Diamonds', 'ID_11'),
 ('10', 'Diamonds', 'ID_12'),
 ('J', 'Diamonds', 'ID_13')]


In [20]:
new_rank_function(lst_sample=h, custom_order=ranks, is_print=True, list_of_straight_ranks=[])

Starting a new straight at index 0 with the element : " 
['4']
Appending the rank 5
Appending the rank 6
Appending the rank 7
Appending the rank 8
Appending the rank 9
Appending the rank 10
Appending the rank J
Here is the long straight ['4', '5', '6', '7', '8', '9', '10', 'J']


[['4', '5', '6', '7', '8', '9', '10', 'J']]

In [21]:
check_lexicographical_straight(lst=h, custom_order=ranks, atleast=4,
                               allow_substraights=True, return_type='card_tuple')

[('4', 'Diamonds', 'ID_1'), ('5', 'Hearts', 'ID_2'), ('6', 'Spades', 'ID_4'), ('7', 'Spades', 'ID_5'), ('8', 'Spades', 'ID_9')]
[('5', 'Hearts', 'ID_2'), ('6', 'Spades', 'ID_4'), ('7', 'Spades', 'ID_5'), ('8', 'Spades', 'ID_9'), ('9', 'Diamonds', 'ID_11')]
[('6', 'Spades', 'ID_4'), ('7', 'Spades', 'ID_5'), ('8', 'Spades', 'ID_9'), ('9', 'Diamonds', 'ID_11'), ('10', 'Diamonds', 'ID_12')]
[('7', 'Spades', 'ID_5'), ('8', 'Spades', 'ID_9'), ('9', 'Diamonds', 'ID_11'), ('10', 'Diamonds', 'ID_12'), ('J', 'Diamonds', 'ID_13')]


[[('4', 'Diamonds', 'ID_1'),
  ('5', 'Hearts', 'ID_2'),
  ('6', 'Spades', 'ID_4'),
  ('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9')],
 [('5', 'Hearts', 'ID_2'),
  ('6', 'Spades', 'ID_4'),
  ('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9'),
  ('9', 'Diamonds', 'ID_11')],
 [('6', 'Spades', 'ID_4'),
  ('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9'),
  ('9', 'Diamonds', 'ID_11'),
  ('10', 'Diamonds', 'ID_12')],
 [('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9'),
  ('9', 'Diamonds', 'ID_11'),
  ('10', 'Diamonds', 'ID_12'),
  ('J', 'Diamonds', 'ID_13')]]

In [22]:
pprint(my_hands['07_interaction_flush_got_2_straights_edgecase_str']['Hand'])

("[('4', 'Diamonds'), ('5', 'Hearts'), ('5', 'Diamonds'), ('6', 'Spades'), "
 "('7', 'Spades'), ('7', 'Hearts'), ('7', 'Clubs'), ('7', 'Diamonds'), ('8', "
 "'Spades'), ('8', 'Diamonds'), ('9', 'Diamonds'), ('10', 'Diamonds'), ('J', "
 "'Diamonds'),]\n")


In [23]:
from functools import partial
def Filtering(e_straight, central_ranks):
    low_rank, high_rank = central_ranks[0], central_ranks[-1]
    if e_straight[0][0] == low_rank or e_straight[-1][0] == high_rank:
        return True

sample_1 =   [[('4', 'Diamonds', 'ID_1'),
             ('5', 'Hearts', 'ID_2'),
             ('6', 'Spades', 'ID_4'),
             ('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9')],
            [('5', 'Hearts', 'ID_2'),
             ('6', 'Spades', 'ID_4'),
             ('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11')],
            [('6', 'Spades', 'ID_4'),
             ('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11'),
             ('10', 'Diamonds', 'ID_12')],
            [('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11'),
             ('10', 'Diamonds', 'ID_12'),
             ('J', 'Diamonds', 'ID_13')],
            [('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11'),
             ('10', 'Diamonds', 'ID_12'),
             ('J', 'Hearts', 'ID_13')]]
sample_2 = [(('4', 'Diamonds', 'ID_1'),
             ('5', 'Hearts', 'ID_2'),
             ('6', 'Spades', 'ID_4'),
             ('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9')),
            (('5', 'Hearts', 'ID_2'),
             ('6', 'Spades', 'ID_4'),
             ('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11')),
            (('6', 'Spades', 'ID_4'),
             ('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11'),
             ('10', 'Diamonds', 'ID_12')),
            (('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11'),
             ('10', 'Diamonds', 'ID_12'),
             ('J', 'Diamonds', 'ID_13')),
            (('7', 'Spades', 'ID_5'),
             ('8', 'Spades', 'ID_9'),
             ('9', 'Diamonds', 'ID_11'),
             ('10', 'Diamonds', 'ID_12'),
             ('J', 'Hearts', 'ID_13'))]
# Invoke the function
func = partial(Filtering, central_ranks=['7', '8'])
result = list(filter(func, sample_2))
pprint(result)

[(('4', 'Diamonds', 'ID_1'),
  ('5', 'Hearts', 'ID_2'),
  ('6', 'Spades', 'ID_4'),
  ('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9')),
 (('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9'),
  ('9', 'Diamonds', 'ID_11'),
  ('10', 'Diamonds', 'ID_12'),
  ('J', 'Diamonds', 'ID_13')),
 (('7', 'Spades', 'ID_5'),
  ('8', 'Spades', 'ID_9'),
  ('9', 'Diamonds', 'ID_11'),
  ('10', 'Diamonds', 'ID_12'),
  ('J', 'Hearts', 'ID_13'))]


## DECONSTRUCTOR

```python
if flushed_suits:
    sy_repo['06_flush_data'].setdefault('flushed_suits', []).extend(flushed_suits)
    sy_repo['06_flush_data'].setdefault('major_suit', major_flush_suit)

    flushed_cards = list(filter(lambda card: card[1] == major_flush_suit, d['sample'] ))
    sy_repo['06_flush_data'].setdefault('flushed_cards', []).extend(flushed_cards)

    flushed_cards_singletons = list(filter(lambda card: card[1] == major_flush_suit, d['sy_repo']['01_singles'].get('All singles', []) ))
    sy_repo['06_flush_data'].setdefault('singletons', []).extend(flushed_cards_singletons)

    flushed_cards_multipletons = list(filter(lambda card: card[1] == major_flush_suit, d['sy_repo']['05_Duplicates'].get('All duplicates', list()) ))
    sy_repo['06_flush_data'].setdefault('multipletons', []).extend(flushed_cards_multipletons)
```

In [24]:
my_dict = {}
my_dict.setdefault('key1', {}).setdefault('sub_key1')
my_dict.get('key1')

{'sub_key1': None}

In [25]:
from collections import Counter
import itertools
from itertools import combinations, product, chain
from functools import partial
def DECONSTRUCTOR(is_random, is_id, edgecase_str=my_hands['01_threequeens_edgecase_flush_str']['Hand'], premium_ranks = ['Q', 'K', 'A']):
    """
    is_random=True paremeter is dominant over edgecase_str, if is_random is False, please pass an 
    edgecase as a string of a list of tuples. 
    
    Returns a dictionary with 4 five-letter keys :
    'ranks' is a nested dictionary with 'e_rank' as a key, with 'length' and 'partition' as subkeys
    'rankw' is ranks_wheel, that is AceLow upto KingHigh
    'suits' is a dictionary with counts of the rank and suit.
    'plebs', and 
    'prems' are lists of tuples that seperate the ranks into plebs and prems, 

    Additionally, 
    'remaining' is a dict of remaining unprocessed that have no premium_pairs
    'prem_pairs' is a dict of lists
    """
    # Create the return object dictionary
    d = dict()
    
    # --------Dealing with parameters---------------------
    if is_random == True and is_id == False:
        thirteen = random.sample(deck, 13)
    elif is_random == False and is_id == False:
        thirteen = ast.literal_eval(edgecase_str) 
    # When is_id is True, use the earlier function
    if is_random == True and is_id == True:
        thirteen = draw_cards_with_IDs(is_random=True)
    elif is_random == False and is_id == True:
        thirteen = draw_cards_with_IDs(is_random=False, edgecase_str=edgecase_str)
   #------------------------------------------------------------
    

    # Count the multiplicities of rank and suit in each card_tuple.
    rank_counter = dict()
    for e_tuple in thirteen:
        # When encountering a new rank, ...
        if e_tuple[0] not in rank_counter.keys():
            # ... append the new rank as a key in the rank_counter dictionary
            rank_counter[e_tuple[0]] = {'length': int(0),'partition': list()}
            # Update the sub_dictionary lengths and partitions
            rank_counter[e_tuple[0]]['partition'].append(e_tuple)
            rank_counter[e_tuple[0]]['length'] += 1
        else:
            rank_counter[e_tuple[0]]['partition'].append(e_tuple)
            rank_counter[e_tuple[0]]['length'] += 1
            
#     suit_counter = Counter(card[1] for card in thirteen)
    suit_counter = dict()
    for e_tuple in thirteen:
        if e_tuple[1] not in suit_counter.keys():
            # ... append the new rank as a key in the rank_counter dictionary
            suit_counter.update({e_tuple[1] : {'length': int(0) , 'partition' : list()}})
            # Update the sub_dictionary lengths and partitions
            suit_counter[e_tuple[1]]['partition'].append(e_tuple)
            suit_counter[e_tuple[1]]['length'] += 1
        else:
            suit_counter[e_tuple[1]]['partition'].append(e_tuple)
            suit_counter[e_tuple[1]]['length'] += 1
            
    # Sorting the ranks and suits counter by the ranks and suits list.
    sorted_ranks_dict = dict(sorted(rank_counter.items(), key=lambda kv: ranks.index(kv[0]), reverse=False))
    sorted_ranks_wheel = dict(sorted(rank_counter.items(), key=lambda kv: ranks_wheel.index(kv[0]), reverse=False))
    
    # Sorting by the suits list
    sorted_suits_dict = dict(sorted(suit_counter.items(), key=lambda kv: suits.index(kv[0]), reverse=False))
    
    # Sorting the partitions by the default(non-wheel) rank ordering.
    for e_suit, suit_data in sorted_suits_dict.items():
        sorted_suits_dict.update({ e_suit : {
                                             'length' : suit_data.get('length'),
                                             'partition': sorted(suit_data.get('partition'), key=lambda tup : ranks.index(tup[0]))
                                             }
                                })

    premiums = [card for card in thirteen if card[0] in premium_ranks]
    plebians = [card for card in thirteen if card[0] not in premium_ranks]
    
    d['sample'] = thirteen
    d.update({'ranks': sorted_ranks_dict})
    d['rankw'] = sorted_ranks_wheel
    d['suits'] = sorted_suits_dict
    d['prems'] = premiums
    d['plebs'] = plebians 
    d['sorted_sample'] = sorted(thirteen, key=lambda card: ( ranks.index(card[0]), suits.index(card[1]) ))
    
    # Synergistic approach - Make note of triple and quads
    sy_repo = {
               '01_singles': {},
               '02_pairs': {},
               '03_trips': {}, 
               '04_quads': {},
               '05_Duplicates':{},
               '06_flush_data' : {},
               '07_straight_data' : {},
               '08_interaction_straights' : {}
              }
    
    for pleb_rank, dict_ in d['ranks'].items():
        length = dict_['length']
        cards = dict_['partition']
        if length == 1:
            sy_repo['01_singles'].setdefault(cards[0][1], []).extend(cards) # Get the first card's suit
            sy_repo['01_singles'].setdefault('All singles', []).extend(cards)
        if length >=2:
            sy_repo['05_Duplicates'].setdefault('All duplicates', []).extend(cards)
            sy_repo['05_Duplicates'].setdefault(cards[0][0], []).append(cards)
        if length == 2:
            sy_repo['02_pairs'].setdefault(pleb_rank, []).append(cards)
        if length == 3:
            # Add the trip itself
            sy_repo['03_trips'].setdefault(pleb_rank, []).append(cards)
            # Add all 3C2 pairs
            for e_pair in combinations(cards, 2):
                sy_repo['02_pairs'].setdefault(pleb_rank, []).append(list(e_pair))
        elif length == 4:
            # Add the quad itself
            sy_repo['04_quads'].setdefault(pleb_rank, []).append(cards)
            # Add all 4C3 trips
            for e_trip in combinations(cards, 3):
                sy_repo['03_trips'].setdefault(pleb_rank, []).append(list(e_trip))
            # Add all 4C2 pairs - too many combos - instead get two complimentary pairs from the quads
            one4C2_combo = 0

            # Split the quads into two pairs - even though quads are superstrong, for the sake of argument...
            first, second, *second_pair = dict_['partition']
            first_pair = [first, second]
            # Append the splitted quads 
            sy_repo['02_pairs'].setdefault(pleb_rank, []).append(first_pair)
            sy_repo['02_pairs'].setdefault(pleb_rank, []).append(second_pair)

    d.update({'sy_repo' : sy_repo})

    # A list of suits with more than 5 cards in 
    flushed_suits = [ suit for suit, suit_data in d['suits'].items() if suit_data['length'] >= 5 ]
    # Get the suit with the most cards
    major_flush_suit = max(d['suits'].items(), key=lambda suit_items : suit_items[1]['length'])[0]

    if flushed_suits:
        sy_repo['06_flush_data'].setdefault('flushed_suits', []).extend(flushed_suits)
        sy_repo['06_flush_data'].setdefault('major_suit', major_flush_suit)
    
        flushed_cards = list(filter(lambda card: card[1] == major_flush_suit, d['sample'] ))
        sy_repo['06_flush_data'].setdefault('flushed_cards', []).extend(flushed_cards)
    
        flushed_cards_singletons = list(filter(lambda card: card[1] == major_flush_suit, d['sy_repo']['01_singles'].get('All singles', []) ))
        sy_repo['06_flush_data'].setdefault('singletons', []).extend(flushed_cards_singletons)
    
        flushed_cards_multipletons = list(filter(lambda card: card[1] == major_flush_suit, d['sy_repo']['05_Duplicates'].get('All duplicates', list()) ))
        sy_repo['06_flush_data'].setdefault('multipletons', []).extend(flushed_cards_multipletons)

    
    list_of_ranks_Ahigh = new_rank_function(lst_sample=d['sample'], 
                                        custom_order=ranks,
                                        list_of_straight_ranks=list(), is_print=False)

    all_straight_ranks = new_rank_function(d['sample'],
                                       custom_order=ranks_wheel,
                                       list_of_straight_ranks=list_of_ranks_Ahigh, is_print=False)
    
    # Finaly, If Ace is part of the longest_continous, we just remove any subsets within the list_of_lists
    filtered = remove_subsets(all_straight_ranks)
    sy_repo['07_straight_data']['01_longest_continuous_ranks'] = filtered

    # Straight-cutter
    atleast=4
    final_subs = []
    for e_long_straight in all_straight_ranks:
        for i in range(len(e_long_straight) - atleast):
            cut_straight = e_long_straight[i : i + atleast + 1]
            if cut_straight not in final_subs:
                final_subs.append(cut_straight)
                
    sy_repo['07_straight_data'].setdefault('03_possible_straight_ranks', []).extend(final_subs)
    

    ## BOTH FLUSH and STRAIGHT PRESENT
    ## CASE 1
    if d['sy_repo']['06_flush_data'] and sy_repo['07_straight_data']['01_longest_continuous_ranks'] :
        print('CASE 1 : BOTH FLUSH AND STRAIGHT PRESENT')        
        # Fetch some variables
        for e_suit in flushed_suits:
            flushed_singletons = sy_repo['06_flush_data'].get('singletons', None)
            flushed_multipletons = sy_repo['06_flush_data'].get('multipletons', None)
            valid_straights = []
            straight_flushes = []
            for e_possible_straight_rank in sy_repo['07_straight_data']['03_possible_straight_ranks']:
                e_valid_straight = []
                for e_rank in e_possible_straight_rank:
                    # use the flushed_singleton, no choice
                    if e_rank in [card[0] for card in flushed_singletons]: # Get the ranks 
                        necessary_singleton  = list(filter(lambda card : card[0]==e_rank, flushed_singletons))
                        e_valid_straight.extend(necessary_singleton)
                    # use a card whose rank is duplicate but whose suit is not the major suit
                    elif e_rank in [card[0] for card in flushed_multipletons]:
                        non_flush_multipleton = list(filter(lambda card : card[0]==e_rank and (card[1] != d['sy_repo']['06_flush_data']['major_suit']), d['sample']))[0]
                        e_valid_straight.append(non_flush_multipleton)
                    else:
                        random_card_not_flush = list(filter(lambda card : card[0]==e_rank, d['sample']))[0]
                        e_valid_straight.append(random_card_not_flush)
                        
                if check_straight_is_straight_flush(straight=e_valid_straight):
                    straight_flushes.append(e_valid_straight)
                else:
                    # Commit the e_valid_straight
                    valid_straights.append(e_valid_straight)
    
            sy_repo['07_straight_data'].setdefault('straights', []).extend(valid_straights)
            sy_repo['06_flush_data'].setdefault('straight_flushes', straight_flushes)
            
        
        # Find the one true_straight 
        if len(sy_repo['07_straight_data']['01_longest_continuous_ranks']) == 1 and (len(sy_repo['07_straight_data']['01_longest_continuous_ranks'][-1]) < 10):
            
            straights = sy_repo['07_straight_data']['straights']
            best_straights2 = list(filter(
            # Only straights whose suit_count equals the mininum number of major_suit among  each other
            lambda s: sum(1 for card in s if card[1] == major_flush_suit) ==
                      min(sum(1 for _, suit, _ in st if suit == major_flush_suit) for st in straights),
            straights
            ))
            # If there is still a tie for minimum, get the 'last' element
            try:
                best_straight=best_straights2[-1]
            except IndexError:
                best_straight=[]
                
            sy_repo['07_straight_data'].setdefault('best_straight', best_straight)

        # FLUSH ANALYSIS
        # Get data
        chosen_straight = d['sy_repo']['07_straight_data'].get('best_straight', list())
        
        ## These list are 2D and need to be flattened
        singleton_flush = d['sy_repo']['06_flush_data'].get('singletons', [])
        multipleton_flush = d['sy_repo']['06_flush_data'].get('multipletons', [])
        flushed_cards = d['sy_repo']['06_flush_data'].get('flushed_cards', [])

        # These are flushed cards that are not in best_straight - already 1D
        pure_flush_cards = [card for card in flushed_cards if card not in chain(chosen_straight)] # Need to flatten the 2D best_straight
        print(f'PURE FLUSH {pure_flush_cards}')
        # Break into 3 non-overallapping lists
        loaned_flushed_cards = [card for card in flushed_cards if card in chain(chosen_straight)] # Need to flatten the 2D best_straight
        print(f"Flush : loaned_flushed_cards in  {e_suit} : {loaned_flushed_cards} ")
        singletons = [card for card in pure_flush_cards if card in singleton_flush]
        print(f"Flush : singletons in {e_suit} : {singletons} ")
        multipletons = [card for card in pure_flush_cards if card in multipleton_flush]
        print(f"Flush : multipletons in {e_suit} : {multipletons} ")

        flushes = []
        straight_flushes = []
        if len(singletons) < 5:
            for e_multipleton_contribution in combinations(multipletons, 5 - len(singletons)):
                single = tuple(singletons.copy())
                # print(e_multipleton_contribution)
                flush = single + e_multipleton_contribution
                # print(f'Diagnose {flush}')
                flush = sorted(flush, key=lambda card: ranks.index(card[0]))
                flushes.append(list(flush))
                
                # if not check_flush_is_straight_flush(flush=flush) and flush not in flushes:
                #     flushes.append(list(flush))
                # elif check_flush_is_straight_flush(flush=flush) and flush not in straight_flushes:
                #     straight_flushes.append(list(flush))
        else:
            for e_combo in list(combinations(singletons, 5))[:12]:
                flush = sorted(e_combo, key=lambda card: ranks.index(card[0]))
                flushes.append(list(flush))
                
                # if not check_flush_is_straight_flush(flush=flush) and flush not in flushes:
                #     flushes.append(list(flush))
                # elif check_flush_is_straight_flush(flush=flush) and flush not in straight_flushes:
                #     straight_flushes.append(list(flush))
        

        sy_repo['06_flush_data'].setdefault('flushes', flushes)
        sy_repo['06_flush_data'].setdefault('straight_flushes', straight_flushes)

    # Case 2 flush-no, straight-yes
    elif not d['sy_repo']['06_flush_data'] and sy_repo['07_straight_data']['01_longest_continuous_ranks']: # If no_flush_present
        print('CASE 2 : FLUSH ABSENT AND STRAIGHT PRESENT')

        # # 1️⃣  Group all cards by rank.
        #     Example: {'5': [5♦, 5♥], '6': [6♠], ...}
        rank_to_cards_meta = defaultdict(list)
        for card in d['sample']:
            rank_to_cards_meta[card[0]].append(card)
                
        # Must consider whether there is more than 1 long_continuous straight
        if len(sy_repo['07_straight_data']['01_longest_continuous_ranks']) == 1:
            print('ONLY 1 Long straight_ranks found')
            
            for e_long_straight_ranks in sy_repo['07_straight_data']['01_longest_continuous_ranks']:
                no_2_straights_flag = False # This bool shows if the possibility of 2 straights existing in a long_straight_run
                # Get the central ranks list
                leading_singles, central_ranks, trailing_singles = get_central_slice(lst=e_long_straight_ranks)
                if not central_ranks:
                    # Go straight to cartesian prod - we can consider all combinations because there inverse relationship between straight_length and duplicate ranks encountered.
                    sy_repo['07_straight_data']['03_possible_straight_ranks'] = []
                    sy_repo['07_straight_data'].setdefault('03_possible_straight_ranks', []).append( e_long_straight_ranks[0:4+1] )
                    sy_repo['07_straight_data'].setdefault('03_possible_straight_ranks', []).append( e_long_straight_ranks[5:9+1] )
                    print(f' {e_long_straight_ranks[0:4+1]}, {e_long_straight_ranks[5:9+1]}')
                    print(f"{sy_repo['07_straight_data']['03_possible_straight_ranks']}")

                    super_long_or_2_long_continuous_straight = True
                    break
                else:
                    super_long_or_2_long_continuous_straight = False
                    # Only append this data if 
                    sy_repo['07_straight_data'].setdefault('02_central_ranks', []).append(central_ranks)
                    # Get the list of non-unique rank cards
                    multipleton_rank_cards = d['sy_repo']['05_Duplicates'].get('All duplicates', list())
                    # Filter in only the non-unique rank cards that are central
                    dup_central_cards = list(filter(lambda card: card[0] in central_ranks, multipleton_rank_cards ))
                    # If no duplicate central cards are found, we can safely deduce that there are no_2_straights 
                    if not dup_central_cards:
                        no_2_straights_flag = True
            
                    # Get the set of multipleton_ranks that are central
                    duplicate_central_ranks = set(card[0] for card in dup_central_cards)
                    
                    for e_rank in central_ranks: # Iterate through the central ranks(expectation)
                        if e_rank not in duplicate_central_ranks: # Every central rank must be in the duplicate_central_ranks(actual), otherwise ...
                            # ... we can abort
                            print(f'Could not find another duplicate rank card for rank :{e_rank} in {duplicate_central_ranks}(actual duplicates) VS {central_ranks}(required duplicates)')
                            print(f'Abort. No 2_straight found in long_straight  ')
                            no_2_straights_flag = True
                            break
                        else: 
                            print(f'Found the rank {e_rank} in {duplicate_central_ranks}(actual duplicates)')
                    else:
                        no_2_straights_flag = False
                        
                if no_2_straights_flag:
                    sy_repo['07_straight_data'].setdefault('Got_2_straights', False)
                    global_limit = 5
                
                    possible_sets = []
                    lengths = []
                
                    # First pass: calculate all lengths
                    for e_possible_straight_ranks in sy_repo['07_straight_data']['03_possible_straight_ranks']:
                        e_cards_lists = [rank_to_cards_meta[rank] for rank in e_possible_straight_ranks]
                        print(f"\nHere is the card lists : \n{e_cards_lists}\n")
                        e_all_possible_straights_per_5_ranks = list(product(*e_cards_lists)) # This is a 2D list ie a list of tuples 
                        e_num_of_possible_straghts_per_5_ranks = len(e_all_possible_straights_per_5_ranks)
                        lengths.append(e_num_of_possible_straghts_per_5_ranks)
                        print(f'num of possible_straights is {e_num_of_possible_straghts_per_5_ranks} for above')
                        possible_sets.append(e_all_possible_straights_per_5_ranks[::-1])  # store reversed

                    print(f'The lengths are {lengths}') # For example :[12, 6, 4, 2] or [12, 4, 4]
                    # print('The possible sets are : \n')
                    # pprint(possible_sets)
                    min_length = min(lengths) # 2 or 4
                    min_indices = [i for i, l in enumerate(lengths) if l == min_length] # In our example it is [3] or [1, 2]
                
                    # Second pass: collect straights from only min-length iterations
                    collected = []
                    if len(min_indices) == 1:
                        print(f'Only one minimum length found.')# For example :[12, 6, 4, 2]
                        # Only one set with min length → take them all
                        idx = min_indices[0]
                        collected.extend(possible_sets[idx][:global_limit])
                    else:
                        # Multiple sets with min length → limit to global_limit
                        for idx in min_indices:
                            for s in possible_sets[idx]:
                                if len(collected) >= global_limit:
                                    break
                                collected.append(s)
                
                    # Store result
                    sy_repo['07_straight_data'].setdefault('straights', []).extend(collected)

                       
                # Overlapping straights           
                else: #not no_2_straights_flag ie Got 2 straights ergo use low-high filtering
                    sy_repo['07_straight_data'].setdefault('Got_2_straights', True)      
                    # print(f"The dup central cards : {dup_central_cards}")
                    # print(f"The central ranks are : {central_ranks}")

                    # Filter obj_1['ranks'] dict for keys in central_ranks
                    filtered_dict = dict(filter(lambda key_value: key_value[0] in central_ranks, d['ranks'].items()))
                    # Create a 3-D list of [[5-pair], [6-pair], [7-pair], ...etc]
                    duplicate_central_cards_2D = []
                    for key, sub_dict in filtered_dict.items():
                        duplicate_central_cards_2D.append(sub_dict['partition'])
                    
                    # Create a 3-D list of [ [almost_straight_1], [almost_straight_1] ]
                    almost_2_straights = list(zip(*duplicate_central_cards_2D))

                    print(f'Diagnose almost :  {almost_2_straights}')

                    # These are 3-D lists -> [[(5Hearts, (5Clubs)], [(),(),()]]
                    leading_cards = [rank_to_cards_meta[rank] for rank in leading_singles]
                    trailing_cards = [rank_to_cards_meta[rank] for rank in trailing_singles]

                     # Create a 3-D list of [ [straight_1], [straight_2] ]
                    try:
                        lower_straight = (leading_cards[0][0],) + almost_2_straights[0]
                        higher_straight = almost_2_straights[-1] + (trailing_cards[-1][-1],)
                    except IndexError: # If there are no leading nor trailing cards,
                        lower_straight = almost_2_straights[0]
                        higher_straight = almost_2_straights[-1]

                    print(f"Diagnose {lower_straight}")

                    sy_repo['07_straight_data'].setdefault('straights', []).append(lower_straight)
                    sy_repo['07_straight_data'].setdefault('straights', []).append(higher_straight)
                    
                    
    
                    
                        
                    # for e_possible_straight_ranks in sy_repo['07_straight_data']['03_possible_straight_ranks']:
                    #     # For each rank, get the list of possible cards,
                    #     # then take the Cartesian product to get every valid combination.
                    #     e_cards_lists = [rank_to_cards_meta[rank] for rank in e_possible_straight_ranks]
                    #     print(f"\nHere is the card lists : \n{e_cards_lists}\n")
                    #     # e_all_possible_straights_per_5_ranks = list(product(*e_cards_lists))
                    #     e_pruned_list = []
                    #     for e_ranks_list in e_cards_lists:
                    #         if len(e_ranks_list) > 1 and e_ranks_list[0][0] not in central_ranks:
                    #             only_card = e_ranks_list[0]
                    #             # Put into a tuple
                    #             only_card_tuple = only_card
                    #             # Put the tuple into a list
                    #             outer_list = []
                    #             outer_list.append(only_card_tuple)
                                
                    #             e_pruned_list.append(outer_list) # [('2', 'Spades', 'ID_1')] into [] 
                    #         else:
                    #             e_pruned_list.append(e_ranks_list[0:1+1]) # Append a max of 2 card for each central rank
                    #     print(f'Pruned card lists\n{e_pruned_list}')
                    #     e_all_pruned_straights_per_5_ranks = list(product(*e_pruned_list))
                    #     print(f'Combinations of\n{e_all_pruned_straights_per_5_ranks}')
                   
                    #     sy_repo['07_straight_data'].setdefault('straights', []).extend(e_all_pruned_straights_per_5_ranks)
                        
                    # # Filter further after loop
                    # else:
                    #     my_func = partial(Filtering, central_ranks=central_ranks)
                    #     results = list(filter(my_func, sy_repo['07_straight_data']['straights']))
                        
                    #     sy_repo['07_straight_data'].update({'straights' : results})


        super_long_or_2_long_continuous_straight = False
        if super_long_or_2_long_continuous_straight or len(sy_repo['07_straight_data']['01_longest_continuous_ranks']) > 1: # Must consider whether there is more than 1 long_contnuous straight or super_long_straight
            if len(sy_repo['07_straight_data']['01_longest_continuous_ranks']) == 1:
                print('Superlong straight found')
            else:
                print('WOW, 2 disjoint long straights found!! ')
            # finally
            for e_possible_straight_ranks in sy_repo['07_straight_data']['03_possible_straight_ranks']:
                e_cards_lists = [rank_to_cards_meta[rank] for rank in e_possible_straight_ranks]
                print(f"\nHere is the card lists : \n{e_cards_lists}\n")
                e_all_possible_straights_per_5_ranks = list(product(*e_cards_lists))
                # print(e_all_possible_straights_per_5_ranks) ## Need to flatten
                for e_straight_tuple in e_all_possible_straights_per_5_ranks[:1]: # Limit to 1 straight per_5_ranks
                    *e_straight_list, = e_straight_tuple
                    sy_repo['07_straight_data'].setdefault('straights', []).append(e_straight_list)
            
    # Case 3 : flush-yes, straight-no
    elif d['sy_repo']['06_flush_data'] and not sy_repo['07_straight_data']['01_longest_continuous_ranks']:
        # Set up an empty key : value_list
        sy_repo['07_straight_data'].setdefault('straights', [])
        print('CASE 3 : FLUSH PRESNET AND STRAIGHT ABSENT')
        # FLUSH ANALYSIS
        # Get data
        chosen_straight = d['sy_repo']['07_straight_data'].get('best_straight', list())
        
        ## These list are 2D and need to be flattened
        singleton_flush = d['sy_repo']['06_flush_data'].get('singletons', [])
        multipleton_flush = d['sy_repo']['06_flush_data'].get('multipletons', [])
        flushed_cards = d['sy_repo']['06_flush_data'].get('flushed_cards', [])

        #These are flushed cards that are not in best_straight - already 1D
        pure_flush_cards = [card for card in flushed_cards if card not in chain(chosen_straight)] # Doesn't make a differenec if use * because chosen_straight is empty
        # print(f'PURE FLUSH {pure_flush_cards}')
        # # Break into 3 non-overallapping lists
        # loaned_flushed_cards = [card for card in flushed_cards if card in chain(*chosen_straight)] # Need to flatten the 2D best_straight
        # print(f"Flush : loaned_flushed_cards in  {e_suit} : {loaned_flushed_cards} ")
        singletons = [card for card in pure_flush_cards if card in singleton_flush]
        print(f"Flush : singletons in {e_suit} : {singletons} ")
        multipletons = [card for card in pure_flush_cards if card in multipleton_flush]
        print(f"Flush : multipletons in {e_suit} : {multipletons} ")

        flushes = []
        straight_flushes = []
        if len(singletons) < 5:
            for e_multipleton_contribution in combinations(multipletons, 5 - len(singletons)):
                single = tuple(singletons.copy())
                # print(e_multipleton_contribution)
                flush = single + e_multipleton_contribution
                # print(f'Diagnose {flush}')
                flush = sorted(flush, key=lambda card: ranks.index(card[0]))
                if not check_flush_is_straight_flush(flush=flush) and flush not in flushes:
                    flushes.append(list(flush))
                elif check_flush_is_straight_flush(flush=flush) and flush not in straight_flushes:
                    straight_flushes.append(list(flush))
        else:
            for e_combo in list(combinations(singletons, 5))[:12]:
                flush = sorted(e_combo, key=lambda card: ranks.index(card[0]))
                if not check_flush_is_straight_flush(flush=flush) and flush not in flushes:
                    flushes.append(list(flush))
                elif check_flush_is_straight_flush(flush=flush) and flush not in straight_flushes:
                    straight_flushes.append(list(flush))
        

        sy_repo['06_flush_data'].setdefault('flushes', flushes)
        sy_repo['06_flush_data'].setdefault('straight_flushes', straight_flushes)
            
    # Case 0 : flush-no, straight-no
    else:
        print('CASE 0 : BOTH FLUSH AND STRAIGHT ABSENT')
    
            
    
    # # Find the longest sub-list
    # try: 
    #     longest_list = max(li_of_lis, key=len)
    #     sy_repo['07_straight_data']['longest_continuous_rank'] = longest_list
    # except TypeError: #TypeError occurs for samples without straights
    #     pass
    
    
    d.update({'sy_repo' : sy_repo})

    # # Two-straights analysis with no flush
    # if len(sy_repo['07_straight_data']['longest_continuous_ranks']) >= 5 and not sy_repo['06_flushed_data']['flushed_suits']:
    #     print(f'Here is the long straight {straight_ranks}')
    #     abort_flag = False
    #     # Get the central ranks list
    #     central_ranks = get_central_slice(lst=straight_ranks)
    #     print(f'Printing the central ranks : {central_ranks} with  this straight_ranks : {straight_ranks}\n')
    #     # Get the list of non-unique rank cards
    #     multipleton_rank_cards = obj_1['sy_repo']['Duplicates']['All duplicates']
    #     # Filter in only the non-unique rank cards that are central
    #     dup_central_cards = list(filter(lambda card: card[0] in central_ranks, multipleton_rank_cards ))
    #     # If no duplicate central cards are found, we can safely abort
    #     if not dup_central_cards:
    #         abort_flag = True

    #     # Get the set of multipleton_ranks that are central
    #     duplicate_central_ranks = set(card[0] for card in dup_central_cards)
        
    #     for e_rank in central_ranks: # Iterate through the central ranks(expectation)
    #         if e_rank not in duplicate_central_ranks: # Every central rank must be in the duplicate_central_ranks(actual), otherwise ...
    #             # ... we can abort
    #             print(f'Could not find a duplicate rank :{e_rank} in {duplicate_central_ranks}(actual duplicates)')
    #             print(f'Abort. ')
    #             abort_flag = True
    #             break
    #         else: 
    #             print(f'Found the rank {e_rank} in {duplicate_central_ranks}(actual duplicates)')
    return d

In [26]:
long_straight_10plus = ranks[:10+1]
central_ranks = get_central_slice(lst=ranks[:9+1])
if not central_ranks:
    # Go straight to cartesian prod
    print(long_straight_10plus[:5], long_straight_10plus[-6:-1])


This is the center index : 5 which maps to element [7] and half value of 0
This is the start index : 5 which maps to this element : 7
This is the end index : 5 for even-length list/csl which maps to 7. Exclusive


In [27]:
ranks[:9+1]

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J']

In [28]:
lengths = [12, 4, 4] # For example :[12, 6, 4, 2] or [12, 4, 4]
min_length = min(lengths) # 2 or 4
min_indices = [i for i, l in enumerate(lengths) if l == min_length] # In our example []
min_indices

[1, 2]

```python
long_straight and fluhs_present:
    need new strategy
long_straight and no_flush_present:
    if got 2_straights: #not no_2_straights # ie check central cards are duplicate
        Get filtered straights # Low & High
    if not 2_straights:
        Expand to all straigths # ie Cartesian product
```

2_straight analysis within a single long run is only valid if no flush is present!

In [29]:
obj_1 = DECONSTRUCTOR(is_random=False, is_id=True, edgecase_str=my_hands['07_interaction_flush_got_2_straights_edgecase_str']['Hand'])
pprint(obj_1)

Here is the long straight ['4', '5', '6', '7', '8', '9', '10', 'J']
CASE 1 : BOTH FLUSH AND STRAIGHT PRESENT
PURE FLUSH [('4', 'Diamonds', 'ID_1'), ('5', 'Diamonds', 'ID_3'), ('7', 'Diamonds', 'ID_8'), ('8', 'Diamonds', 'ID_10'), ('10', 'Diamonds', 'ID_12'), ('J', 'Diamonds', 'ID_13')]
Flush : loaned_flushed_cards in  Diamonds : [('9', 'Diamonds', 'ID_11')] 
Flush : singletons in Diamonds : [('4', 'Diamonds', 'ID_1'), ('10', 'Diamonds', 'ID_12'), ('J', 'Diamonds', 'ID_13')] 
Flush : multipletons in Diamonds : [('5', 'Diamonds', 'ID_3'), ('7', 'Diamonds', 'ID_8'), ('8', 'Diamonds', 'ID_10')] 
{'plebs': [('4', 'Diamonds', 'ID_1'),
           ('5', 'Hearts', 'ID_2'),
           ('5', 'Diamonds', 'ID_3'),
           ('6', 'Spades', 'ID_4'),
           ('7', 'Spades', 'ID_5'),
           ('7', 'Hearts', 'ID_6'),
           ('7', 'Clubs', 'ID_7'),
           ('7', 'Diamonds', 'ID_8'),
           ('8', 'Spades', 'ID_9'),
           ('8', 'Diamonds', 'ID_10'),
           ('9', 'Diamonds', 'ID_

## Generate 3 monsters

In [30]:
import numpy as np
import itertools
from functools import reduce
def generate_three_monsters(enable_print=True, enable_custom_monsters=True, custom_monsters=['trips', 'straights', 'straights']):
    """
    Generates 3 monster hands

    Defaults to ['trips', 'straights', 'straights']

    Otherwise set enable_custom_monsters=False for random 3 monsters.
    """
    
    low_ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10']
    monsters = ['trips', 'straights', 'flushes', 'fullhouses', 'straight_flushes', 'quads'] # Straight_flush ahead of quads
    # Keeping track of the reamining cards
    my_deck = deck.copy()
    # Keeping track of the remaining ranks 
    available_ranks = ranks.copy()
    # Memoize quad_ranks to prevent 5, 10
    quad_ranks = []
    # The final thirteen cards
    my_thirteen = []
    # Inverse of my_deck
    memoized_cards = []
    # Keep track of cards that can cause promotion
    promoters = []

    if enable_custom_monsters:
        my_monsters = custom_monsters
    else:
        first_monster = ['trips', 'quads']
        first_monster = np.random.choice(first_monster, size=1, replace=False, p=[0.99, 0.01]).tolist()
        if first_monster[0] == 'quads':
            two_monsters = ['straight_flushes', 'quads']
            my_monsters = [*first_monster, *two_monsters]
        elif first_monster[0] == 'trips':
            two_monsters = np.random.choice(monsters, size=2, replace=True).tolist()
            two_monsters = sorted(two_monsters, key=lambda el: monsters.index(el))
            my_monsters = [*first_monster, *two_monsters]
        
    for e_monster in my_monsters:
        if e_monster == 'trips':
            # Get a random rank from the ranks that haven't been chosen
            another_random_rank = ''.join([str(one_rank) for one_rank in random.sample(available_ranks, 1)])
            # Generate the triples
            another_triple = random.sample([card for card in my_deck if card[0] == another_random_rank], 3) 
            # Update available_ranks for future triples or fullhouses
            available_ranks.pop(available_ranks.index(another_random_rank))
            # Clean the deck - discard some cards
            for e_card in another_triple:
                my_deck.pop(my_deck.index(e_card))
                memoized_cards.append(e_card)
            # Append the trips
            my_thirteen.append(another_triple) 
            # Promoter 
            promoter = random.sample([card for card in my_deck if card[0] == another_random_rank and card not in another_triple], 1)
            for e_card in promoter:
                promoters.append(e_card)
        elif e_monster == 'straights':        
            while True:
                # Regenerate the straight
                generated_straight = []
                promoters = [] # Reset promoters after each loop
                # Choose a rank to be the low in the straight - can be any rank
                low_rank = ''.join([str(one_rank) for one_rank in random.sample(low_ranks, 1)])
                if low_rank == 'A': # If the randomly chosen rank is 'A' - use rank_wheel instead
                    # Get the ranks that will form the straight, add an Ace for this case
                    straight_ranks = ranks_wheel[low_ranks.index(low_rank) : low_ranks.index(low_rank)+4+1] # [0(A), 1(2), ..., 9(10), 10(J), 11(Q), 12(K)]
                    promoter_rank = ranks_wheel[low_ranks.index(low_rank)+5]
                    # Get all promoting cards
                    promoter = [(promoter_rank, suit) for suit in suits]
                    for e_card in promoter:
                        promoters.append(e_card)
                else: # Use ranks instead
                    straight_ranks = ranks[low_ranks.index(low_rank)-1 : low_ranks.index(low_rank)+3+1] # [0(2), 1(3), ...8(10), 9(J), 10(Q), 11(K), 12(A)]
                    try: # In the event of drawing a '10', there will be no promoter_rank
                        promoter_rank = ranks[low_ranks.index(low_rank)+4]
                       # Get all promoting cards
                        promoter = [(promoter_rank, suit) for suit in suits]
                        for e_card in promoter:
                            promoters.append(e_card)
                    except IndexError:
                        pass 
                try: # Attempt to pick cards for e_rank
                    for e_rank in straight_ranks:
                        candidates = [card for card in my_deck if card[0] == e_rank] # These are the available cards that e_rank can take.
                        if not candidates:
                            raise ValueError(f'No cards left for that rank {e_rank}')
                        # Get a random suit to go with the rank as long as it is available in my_deck
                        generated_card = random.choice(candidates)
                        generated_straight.append(generated_card)
                    # genuine_straight = list(itertools.chain.from_iterable(generated_straight))
                    if not check_straight_is_straight_flush(straight=generated_straight): # If given straight is not a straight_flush then exit
                        break
                    else:
                        continue
                except ValueError:
                    continue
            # Clean the deck - discard some cards
            for e_card in generated_straight:
                my_deck.pop(my_deck.index(e_card))
                memoized_cards.append(e_card)
            # Update hand
            my_thirteen.append(generated_straight)
        elif e_monster == 'flushes':
            while True:
                # Get a random_suit
                my_suit = ''.join([str(one_rank) for one_rank in random.sample(suits, 1)])
                # Get 5 cards of a single suit
                five_of_a_suit = random.sample([card for card in my_deck if card[1] == my_suit], 5)
                # Get a list of the rank combinations in the flush
                flush_ranks = [card[0] for card in five_of_a_suit]
                # Get the highest rank
                highest_rank = reduce(lambda x,y : x if ranks.index(x) > ranks.index(y) else y, flush_ranks)
                # Get the promoting ranks
                promoting_ranks = ranks[ranks.index(highest_rank)+1:None]
                promoter = [(e_rank, my_suit) for e_rank in promoting_ranks]
                for e_card in promoter:
                    promoters.append(e_card)
                if not check_flush_is_straight_flush(flush=five_of_a_suit):
                    break
            # Clean the deck - discard some cards
            for e_card in five_of_a_suit:
                my_deck.pop(my_deck.index(e_card))
                memoized_cards.append(e_card)
            # Update hand
            my_thirteen.append(five_of_a_suit)
        elif e_monster == 'fullhouses':
            # Get a random rank from the ranks that haven't been chosen
            two_random_ranks = random.sample(available_ranks, 2)
            # Update available_ranks for future triples or fullhouses
            for rank in two_random_ranks:
                available_ranks.pop(available_ranks.index(rank))
            
            kids = random.sample([card for card in my_deck if card[0] == two_random_ranks[0]], 3)
            parents = random.sample([card for card in my_deck if card[0] == two_random_ranks[1]], 2)
            fullhouse = [*kids, *parents]
    
            promoter_1 = random.sample([card for card in my_deck if card[0] == two_random_ranks[0] and card[0] in two_random_ranks[0] ], 1) 
            promoter_2 = random.sample([card for card in my_deck if card[0] == two_random_ranks[1] and card[0] in two_random_ranks[1] ], 2) 
            promoter = [*promoter_1, *promoter_2]
            for e_card in promoter:
                promoters.append(e_card)
            # Clean the deck - discard some cards
            for e_card in fullhouse:
                my_deck.pop(my_deck.index(e_card))
                memoized_cards.append(e_card)
            # Update hand
            my_thirteen.append(fullhouse)
        elif e_monster == 'quads':
            while True:
                # Get a tentative rank
                quad_rank = ''.join(random.sample(available_ranks, 1))
                try:
                    # In the event that the chosen quad_rank is part of other lower monsters....
                    quads = random.sample([card for card in my_deck if card[0] == quad_rank], 4)
                    break
                except ValueError:
                    continue # ... Restart while 
            # Clean the deck - discard some cards
            for e_card in quads:
                my_deck.pop(my_deck.index(e_card))
                memoized_cards.append(e_card)
            # Update hand
            my_thirteen.append(quads)
        elif e_monster == 'straight_flushes':
            while True:
                while_flag = False
                # Regenerate the straight
                generated_straight = []
                candidate_straight = [] 
                # Choose a rank to be the low in the straight - can be any rank from A-10
                low_rank = ''.join([str(one_rank) for one_rank in random.sample(low_ranks, 1)])
                # Get a random_suit
                my_suit = ''.join([str(one_rank) for one_rank in random.sample(suits, 1)])
                if low_rank == 'A': # # If the randomly chosen rank is 'A' - use rank_wheel instead
                    # Get the ranks that will form the straight, add an Ace for this case
                    straight_ranks = ranks_wheel[low_ranks.index(low_rank) : low_ranks.index(low_rank)+5] # [0(A), 1(2), ..., 9(10), 10(J), 11(Q), 12(K)]
                    promoter_rank = ranks_wheel[low_ranks.index(low_rank)+5]
                    # Get the single promoting card
                    promoter = (promoter_rank, my_suit)
                else: # Use ranks instead
                    straight_ranks = ranks[low_ranks.index(low_rank)-1 : low_ranks.index(low_rank)+4] # [0(2), 1(3), ..., 9(J), 10(Q), 11(K), 12(A)]
                    try: # In the event of drawing a '10', there will be no promoter_rank           
                        promoter_rank = ranks[low_ranks.index(low_rank)+4]
                        # Get the tentative single promoting card
                        promoter = (promoter_rank, my_suit)
                    except IndexError:
                        pass           
                for e_rank in straight_ranks:
                    # print(f'Dealing with e_rank : {e_rank}')
                    # Get the suit to go with the rank as long as it is available in my_deck
                    generated_card = [card for card in deck if card[0] == e_rank and card[1] == my_suit]
                    generated_straight.append(generated_card)
                candidate_straight = list(itertools.chain.from_iterable(generated_straight))
                # Validate the candidate_straight
                for card in candidate_straight:
                    if card in memoized_cards:
                        print(f"{candidate_straight} is rejected because of {card}")
                        while_flag = True # Toggle true
                        break # From for-loop
                    else:
                        pass # Do nothing
                if while_flag: # If true revert back to start of the while loop and restart process
                    continue
                else:
                    # Candidate straight is ready to be appended, along with its promoter
                    my_thirteen.append(candidate_straight)
                    try:
                        # In the event of drawing a 10 as the low-rank, there is no promoter
                        print(f'Promoting the straight-flush : {promoter}')
                        promoters.append(promoter)
                    except UnboundLocalError: # UnboundLocalError: local variable 'promoter' referenced before assignment
                        print(f'A 10 was the low-rank')
                        pass
                    # Update the available ranks for easier processing of quads
                    for e_card in candidate_straight:
                        try: # In the event of that the rank was already removed by trips, or fullhouses....
                            available_ranks.pop(available_ranks.index(e_card[0]))
                        except ValueError:
                            pass # ... Do nothing ie the rank,e_card[0] is a rank that we have already removed before by previous trips, fullhouses, straight_flushes
                    break
            # Clean the deck - discard some cards
            for e_card in candidate_straight:
                try:
                    my_deck.pop(my_deck.index(e_card))
                except ValueError:
                    print(f'Already discarded {e_card}')
                memoized_cards.append(e_card)
                
    flat_thirteenish = list(itertools.chain.from_iterable(my_thirteen))
    slots_left = 13 - len(flat_thirteenish)
    memoizied_npc_cards = []
    npc_cards = None
    
    while True:
        if slots_left > 0:
            # Get as many cards as needed
            npc_cards = random.sample([card for card in my_deck if card not in [*memoized_cards, *promoters]], slots_left)
            # Isolate ranks
            npc_ranks = [card[0] for card in npc_cards]
            # Count ranks
            npc_rank_counts = {rank: npc_ranks.count(rank) for rank in set(npc_ranks)}
            # Check if any rank appears 2 or more times (i.e., preventing pairs forming by the npcs)
            if any(count >= 2 for count in npc_rank_counts.values()):
                continue
            else:
                official_thirteen = [*flat_thirteenish, *npc_cards]
                break
        elif slots_left == 0:
            official_thirteen = flat_thirteenish
            break

    if enable_print:
        print(f" My monsters ->{my_monsters}")    
        print(f"My thirteen -")
        from pprint import pprint 
        pprint(my_thirteen)
        print(f'The promoters :')
        pprint(promoters)
        print(f"The npcs :")
        pprint(npc_cards)
        print(f'Final thirteen')
        pprint(sorted(official_thirteen, key=lambda card : (ranks.index(card[0]), card[1] )))

    return str(official_thirteen)

In [31]:
# Invoke the function
generate_three_monsters(enable_print=True, enable_custom_monsters=False)

 My monsters ->['trips', 'trips', 'fullhouses']
My thirteen -
[[('3', 'Hearts'), ('3', 'Diamonds'), ('3', 'Clubs')],
 [('5', 'Diamonds'), ('5', 'Hearts'), ('5', 'Spades')],
 [('K', 'Diamonds'),
  ('K', 'Hearts'),
  ('K', 'Clubs'),
  ('7', 'Diamonds'),
  ('7', 'Spades')]]
The promoters :
[('3', 'Spades'),
 ('5', 'Clubs'),
 ('K', 'Spades'),
 ('7', 'Hearts'),
 ('7', 'Spades')]
The npcs :
[('4', 'Spades'), ('8', 'Diamonds')]
Final thirteen
[('3', 'Clubs'),
 ('3', 'Diamonds'),
 ('3', 'Hearts'),
 ('4', 'Spades'),
 ('5', 'Diamonds'),
 ('5', 'Hearts'),
 ('5', 'Spades'),
 ('7', 'Diamonds'),
 ('7', 'Spades'),
 ('8', 'Diamonds'),
 ('K', 'Clubs'),
 ('K', 'Diamonds'),
 ('K', 'Hearts')]


"[('3', 'Hearts'), ('3', 'Diamonds'), ('3', 'Clubs'), ('5', 'Diamonds'), ('5', 'Hearts'), ('5', 'Spades'), ('K', 'Diamonds'), ('K', 'Hearts'), ('K', 'Clubs'), ('7', 'Diamonds'), ('7', 'Spades'), ('4', 'Spades'), ('8', 'Diamonds')]"

In [32]:
pprint(my_hands)

{'01_threequeens_edgecase_flush_str': {'Hand': "[('Q', 'Spades'), ('Q', "
                                               "'Diamonds'), ('2', "
                                               "'Diamonds'), ('4', "
                                               "'Diamonds'), ('6', "
                                               "'Diamonds'), ('8', "
                                               "'Diamonds'), ('Q', 'Hearts'), "
                                               "('K', 'Spades'), ('A', "
                                               "'Clubs'), ('3', 'Clubs'), "
                                               "('6', 'Hearts'), ('9', "
                                               "'Spades'), ('J', 'Hearts')]",
                                       'Remarks': 'Contains 3 Queens where the '
                                                  'the Queen of Diamonds is '
                                                  'part of a 5 card flush. No '
                               

In [33]:
from pprint import pprint
obj_1 = DECONSTRUCTOR(is_random=False, is_id=True,
                      edgecase_str=generate_three_monsters(enable_print=True, enable_custom_monsters=True, custom_monsters=['trips', 'straights', 'flushes']))
pprint(obj_1)

 My monsters ->['trips', 'straights', 'flushes']
My thirteen -
[[('7', 'Hearts'), ('7', 'Spades'), ('7', 'Clubs')],
 [('10', 'Diamonds'),
  ('J', 'Diamonds'),
  ('Q', 'Clubs'),
  ('K', 'Hearts'),
  ('A', 'Diamonds')],
 [('3', 'Diamonds'),
  ('7', 'Diamonds'),
  ('4', 'Diamonds'),
  ('2', 'Diamonds'),
  ('8', 'Diamonds')]]
The promoters :
[('9', 'Diamonds'),
 ('10', 'Diamonds'),
 ('J', 'Diamonds'),
 ('Q', 'Diamonds'),
 ('K', 'Diamonds'),
 ('A', 'Diamonds')]
The npcs :
None
Final thirteen
[('2', 'Diamonds'),
 ('3', 'Diamonds'),
 ('4', 'Diamonds'),
 ('7', 'Clubs'),
 ('7', 'Diamonds'),
 ('7', 'Hearts'),
 ('7', 'Spades'),
 ('8', 'Diamonds'),
 ('10', 'Diamonds'),
 ('J', 'Diamonds'),
 ('Q', 'Clubs'),
 ('K', 'Hearts'),
 ('A', 'Diamonds')]
Here is the long straight ['10', 'J', 'Q', 'K', 'A']
CASE 1 : BOTH FLUSH AND STRAIGHT PRESENT
PURE FLUSH [('3', 'Diamonds', 'ID_9'), ('7', 'Diamonds', 'ID_10'), ('4', 'Diamonds', 'ID_11'), ('2', 'Diamonds', 'ID_12'), ('8', 'Diamonds', 'ID_13')]
Flush : loaned

## test_dim

In [34]:
def test_dim(testlist, dim=0):
   """tests if testlist is a list and how many dimensions it has
   returns -1 if it is no list at all, 0 if list is empty 
   and otherwise the dimensions of it"""
   if isinstance(testlist, (list, tuple)):
      if testlist == []:
          return dim
      dim = dim + 1
      dim = test_dim(testlist[0], dim)
      return dim
   else:
      if dim == 0:
          return -1
      else:
          return dim
a=[]
print('a', test_dim(a))

a=""
test_dim(a)
print('b', test_dim(a))

a=["A"]
print('c', test_dim(a))

a=["A", "B", "C"]
print('d', test_dim(a))

a=[[1,2,3],[1,2,3]]
print('e', test_dim(a))

a=[[[1,2,3],[4,5,6]], [[1,2,3],[4,5,6]], [[1,2,3],[4,5,6]]]
print('f', test_dim(a))

a=[([1,2,3],[4,5,6]), ([1,2,3],[4,5,6]), ([1,2,3],[4,5,6])]
print('d', test_dim(a))

a 0
b -1
c 1
d 1
e 2
f 3
d 3


In [35]:
straights = [('8', 'Clubs', 'ID_13'), ('9', 'Diamonds', 'ID_9'), ('10', 'Diamonds', 'ID_7'), ('J', 'Hearts', 'ID_1'), ('Q', 'Diamonds', 'ID_3')]
test_dim(testlist=straights, dim=0)

2

## Gropuer

In [36]:
my_dict = {'A1': 1, 'B' : 2}
my_dict.get('A', my_dict.get('B'))

2

In [37]:
# Create the data structure
from itertools import combinations
from collections import defaultdict

def grouper3(obj1):
    pairs = {}
    trips = {}
    quads = {}

    # Step 1️⃣: Build dictionaries for pairs, trips, quads
    for handtype, info in obj1['sy_repo'].items():
        if handtype in ['01_singles', '05_Duplicates', '06_flush_data', '07_straight_data', '08_interaction_straights']:
            pass
        else:
            count = 1
            for rank, combos in info.items():
                for new_index, e_combo in enumerate(combos, 1):
                    label = f"{rank * {'02_pairs': 2, '03_trips': 3, '04_quads': 4}[handtype]}"
                    # Remove prefix from handtype
                    suffix_removed = re.search(pattern=r'^\d{2}_([\w]+)', string=handtype)
                    entry = {f"{suffix_removed.group(1)}_combo{count}_{label}": e_combo}
                    count += 1
                    if handtype == '02_pairs':
                        pairs.update(entry)
                    elif handtype == '03_trips':
                        trips.update(entry)
                    elif handtype == '04_quads':
                        quads.update(entry)

    # Step 2️⃣: Extract prem pairs (top, mid, bot)
    # top_rank = obj1['top'][0][0] if obj1['top'] else ""
    # mid_rank = obj1['mid'][0][0] if obj1['mid'] else ""
    # bot_rank = obj1['bot'][0][0] if obj1['bot'] else ""

    # Step 3️⃣: Build dictionaries for straights, flushes, straight-flushes
    # ✅ Keep up to 2 straights per final rank
    # straight_by_rank = defaultdict(list) # Specify a default value of empty list for new_keys
    
    # Is it using best_straight or a lists of straights? Answer both have the same list dimensions
    straights = obj_1['sy_repo']['07_straight_data'].get('best_straight',
                                                         obj_1['sy_repo']['07_straight_data'].get('straights', 
                                                                                                 list()) )
    # print(f'Diagnose {straights}')
   
    if test_dim(straights) == 2: # For the best straight ie 2D list of 1 element - [(card1), (card2), ..., (rank,suit,ID), (card5)]
        i=1
        key = straights[-1][0]
        straight_by_rank = {}
        straight_by_rank[f'straights_combo{i}_{key}'] = straights
    
    else: # 3D list -> [[(card1),...,(card5)], [(card1), ...(card5)], [e_straight]]
        straight_by_rank = defaultdict(list) # Specify a default value of empty list for new_keys
        for i, e_straight in enumerate(straights, start=1): # For 2D lists with >1 sub_straights : [ [(card1), (card2), ..., (card5)], [(), ..., ()], [()]]
            key = e_straight[-1][0]  # use final card's rank
            straight_by_rank[f'straights_combo{i}_{key}'].extend(e_straight) # Must only use extend, cannot use append!

    limited_straights = {**straight_by_rank}
    # for key, combos in straight_by_rank.items():
    #     # for combo in combos[:2+1]:  # Only up to 2 per rank
    #     #     label, cards = combo
    #         limited_straights[key] = combos

    flushes = obj_1['sy_repo']['06_flush_data'].get('flushes', list())
    limited_flushes = {f'flushes_combo{i}_{value[-1][0]}': value for i, value in enumerate(flushes, start=1)}
    
    # print(f"limited_flushes {limited_flushes}\n\n")
    straight_flushes_data = obj_1['sy_repo']['06_flush_data'].get('straight_flushes', [])
    straight_flushes = {f'straight_flushes_combo{i}_{value[-1][0]}': value
                        for i, value in enumerate(straight_flushes_data, start=1)}

    # Step 4️⃣: Aggregate all valid entries
    data_structure = {
        # f"top_prem_pair_{top_rank*2}": obj1['top'],
        # f"mid_prem_pair_{mid_rank*2}": obj1['mid'],
        # f"bot_prem_pair_{bot_rank*2}": obj1['bot'],
        **pairs, **trips, **quads, **limited_straights, **limited_flushes, **straight_flushes
    }
    
    d = {}
    # Step 5️⃣: Filter out None and prepare data for set comparison
    filtered_data = {k: v for k, v in data_structure.items() if v}
    print(f"Registered units : \n")
    pprint(filtered_data)
    items2 = [(k, {card[2] for card in v}, len(v)) for k, v in filtered_data.items()]

    # Step 6️⃣: Find mutually exclusive groupings with memoization
    mutually_exclusive_groupings = []
    memo = set()

    for r in range(2, len(items2) + 1):
        for combo in combinations(items2, r):
            sets_only = [s for _, s, _ in combo]
            lengths = [l for _, _, l in combo]
            all_ids = set.union(*sets_only)
            total_size = sum(lengths)

            if len(all_ids) == total_size:
                group_keys = [k for k, _, _ in combo]
                structure_label = frozenset((k.split('_')[0], k.split('_')[-1]) for k in group_keys)
                if structure_label in memo and len(group_keys) <=2 :
                    # print(f'Already memoized {structure_label}')
                    continue
                else:
                    memo.add(structure_label)
                    mutually_exclusive_groupings.append(group_keys)
                    # print(f'Adding  {structure_label} to the memo')

  
    # Deduplicate and sort groupings by keys
    unique_groupings_keys = sorted(set(tuple(sorted(group)) for group in mutually_exclusive_groupings),
                                   key=lambda x: (len(x), x))
    
    # Now build value-based version
    unique_groupings_values = []
    for group in unique_groupings_keys:
        values = tuple(data_structure[key] for key in group)
        unique_groupings_values.append(values)

    mapped_dict = dict(zip(unique_groupings_keys, unique_groupings_values))
    
    max_cards_used = 0
    max_keys = 0
    best_hands_dict = {}
    count = 1
    for key_tuple, tup_of_list_cardtuples in mapped_dict.items():
        cur_cards_used = sum(len(lst) for lst in tup_of_list_cardtuples if lst is not None)
        cur_max_keys = len(key_tuple)
        if cur_cards_used > max_cards_used : # and cur_max_keys > max_keys
            # Found a new max, reset
            max_cards_used = cur_cards_used
            # max_keys = cur_max_keys
            # Redefine best_hands_dict
            best_hands_dict = {f'best_hand_{count}': {key_tuple: tup_of_list_cardtuples}}
            count = 1
        elif cur_cards_used == max_cards_used : # and cur_max_keys == max_keys
            best_hands_dict[f'best_hand_{count}'] = {key_tuple: tup_of_list_cardtuples}
            count += 1
    
    d.update({'all_dicts': mapped_dict, 'best_hands': best_hands_dict})

    return d

In [38]:
ali = []
straights = obj_1['sy_repo']['07_straight_data'].get('best_straight_typo', obj_1['sy_repo']['07_straight_data']['straights'] )
if test_dim(straights) == 1:
    ali = straights
else:
    ali = straights
ali

[[('10', 'Diamonds', 'ID_4'),
  ('J', 'Diamonds', 'ID_5'),
  ('Q', 'Clubs', 'ID_6'),
  ('K', 'Hearts', 'ID_7'),
  ('A', 'Diamonds', 'ID_8')]]

## Generate new hands & update `my_hands`

In [39]:
high_integer

18

In [40]:
generate_4_of_a_kind_edgecase_str = generate_four_of_a_kind_edgecase(is_random=True, is_exclude_other_prems=False)
generate_curious_edgecase_str = generate_curious_edgecase()
generate_3_monsters = generate_three_monsters(enable_custom_monsters=True, custom_monsters=['trips', 'straights', 'flushes'])

my_hands.update({f'{high_integer:02d}_generate_4_of_a_kind_edgecase_str': {'Hand' : generate_4_of_a_kind_edgecase_str, 'Remarks': None},   
                     f'{high_integer+1:02d}_generate_curious_edgecase_str': {'Hand' : generate_curious_edgecase_str, 'Remarks': None},
                    f'{high_integer+2:02d}_generate_3_monsters': {'Hand' : generate_3_monsters, 'Remarks': None}
                })

 My monsters ->['trips', 'straights', 'flushes']
My thirteen -
[[('5', 'Spades'), ('5', 'Clubs'), ('5', 'Hearts')],
 [('4', 'Diamonds'),
  ('5', 'Diamonds'),
  ('6', 'Diamonds'),
  ('7', 'Spades'),
  ('8', 'Diamonds')],
 [('A', 'Spades'),
  ('4', 'Spades'),
  ('3', 'Spades'),
  ('9', 'Spades'),
  ('K', 'Spades')]]
The promoters :
[('9', 'Spades'), ('9', 'Hearts'), ('9', 'Diamonds'), ('9', 'Clubs')]
The npcs :
None
Final thirteen
[('3', 'Spades'),
 ('4', 'Diamonds'),
 ('4', 'Spades'),
 ('5', 'Clubs'),
 ('5', 'Diamonds'),
 ('5', 'Hearts'),
 ('5', 'Spades'),
 ('6', 'Diamonds'),
 ('7', 'Spades'),
 ('8', 'Diamonds'),
 ('9', 'Spades'),
 ('K', 'Spades'),
 ('A', 'Spades')]


In [41]:
pprint(my_hands)

{'01_threequeens_edgecase_flush_str': {'Hand': "[('Q', 'Spades'), ('Q', "
                                               "'Diamonds'), ('2', "
                                               "'Diamonds'), ('4', "
                                               "'Diamonds'), ('6', "
                                               "'Diamonds'), ('8', "
                                               "'Diamonds'), ('Q', 'Hearts'), "
                                               "('K', 'Spades'), ('A', "
                                               "'Clubs'), ('3', 'Clubs'), "
                                               "('6', 'Hearts'), ('9', "
                                               "'Spades'), ('J', 'Hearts')]",
                                       'Remarks': 'Contains 3 Queens where the '
                                                  'the Queen of Diamonds is '
                                                  'part of a 5 card flush. No '
                               

## Invoking the main workflow

In [42]:
%%time
# Trying to invoke #generate_3_monsters(enable_custom_monsters=True, custom_monsters=['trips', 'straights', 'flushes'])

obj_1 = DECONSTRUCTOR(is_random=False, is_id=True,
                      edgecase_str=my_hands['14_Case2_all_substraights_have_duplicates']['Hand'])

pprint(obj_1)
obj_2 = grouper3(obj_1) # grouper(obj_2)
print(my_hands['14_Case2_all_substraights_have_duplicates']['Remarks'])

Here is the long straight ['2', '3', '4', '5', '6', '7', '8', '9']
CASE 2 : FLUSH ABSENT AND STRAIGHT PRESENT
ONLY 1 Long straight_ranks found

This is the center index : 4 which maps to element [6] and half value of 1
This is the start index : 3 which maps to this element : 5
This is the end index : 5 for even-length list/csl which maps to 7. Exclusive
Could not find another duplicate rank card for rank :5 in set()(actual duplicates) VS ['5', '6'](required duplicates)
Abort. No 2_straight found in long_straight  

Here is the card lists : 
[[('2', 'Spades', 'ID_1'), ('2', 'Hearts', 'ID_2')], [('3', 'Clubs', 'ID_3'), ('3', 'Diamonds', 'ID_4'), ('3', 'Hearts', 'ID_5')], [('4', 'Spades', 'ID_6'), ('4', 'Hearts', 'ID_7')], [('5', 'Clubs', 'ID_8')], [('6', 'Diamonds', 'ID_9')]]

num of possible_straights is 12 for above

Here is the card lists : 
[[('3', 'Clubs', 'ID_3'), ('3', 'Diamonds', 'ID_4'), ('3', 'Hearts', 'ID_5')], [('4', 'Spades', 'ID_6'), ('4', 'Hearts', 'ID_7')], [('5', 'Clubs'

In [43]:
my_hands['13_Case2_one_straight_has_no_dupilcate_ranks']['Remarks']

'Needed to improve test_dim function with tuple, so each straight in Case 2 is registered properly.'

In [44]:
obj_2['best_hands']

{'best_hand_6': {('pairs_combo1_22',
   'pairs_combo5_44',
   'straights_combo1_9',
   'trips_combo1_333'): ([('2', 'Spades', 'ID_1'), ('2', 'Hearts', 'ID_2')],
   [('4', 'Spades', 'ID_6'), ('4', 'Hearts', 'ID_7')],
   [('5', 'Clubs', 'ID_8'),
    ('6', 'Diamonds', 'ID_9'),
    ('7', 'Spades', 'ID_10'),
    ('8', 'Clubs', 'ID_12'),
    ('9', 'Diamonds', 'ID_13')],
   [('3', 'Clubs', 'ID_3'),
    ('3', 'Diamonds', 'ID_4'),
    ('3', 'Hearts', 'ID_5')])},
 'best_hand_1': {('pairs_combo1_22',
   'pairs_combo5_44',
   'straights_combo2_9',
   'trips_combo1_333'): ([('2', 'Spades', 'ID_1'), ('2', 'Hearts', 'ID_2')],
   [('4', 'Spades', 'ID_6'), ('4', 'Hearts', 'ID_7')],
   [('5', 'Clubs', 'ID_8'),
    ('6', 'Diamonds', 'ID_9'),
    ('7', 'Spades', 'ID_10'),
    ('8', 'Hearts', 'ID_11'),
    ('9', 'Diamonds', 'ID_13')],
   [('3', 'Clubs', 'ID_3'),
    ('3', 'Diamonds', 'ID_4'),
    ('3', 'Hearts', 'ID_5')])}}

## If fantasyland is achieved, then showcase

In [45]:
def num_double_pairs(n):
    if n in (0, 1):
        return 0
    elif n in (2, 3):
        return 1
    elif n in (4, 5):
        return 2
    elif n in (6, 7):
        return 3
    elif n in (8, 9):
        return 4
    elif n in (10, 11):
        return 5
    else:
        raise ValueError("n out of expected range")

In [46]:
max_cards_used = 0
max_keys = 0
fantasy_hands_dict = {}
count = 1
for key_tuple, tup_of_list_cardtuples in obj_2['all_dicts'].items():
    prem_pairs_count = 0
    prem_trips_count = 0
    pleb_pairs_count = 0
    regular_trips_count = 0
    monsters_count = 0
    found_flag = False
    found_prem_pair = False

    has_prem_pair = any(re.search(pattern=r'^(pairs).*(Q{2,3}|K{2,3}|A{2,3})$', string=e_combo) for e_combo in key_tuple)
    has_pleb_pair = any(re.search(pattern=r'^pairs.*_(22|33|44|55|66|77|88|99|1010|JJ|TT)$', string=e_combo) for e_combo in key_tuple)
    
    if has_prem_pair:
        for e_combo in key_tuple:
            prem_pairs_match = re.search(pattern=r'^pairs.*(QQ|KK|AA)$', string=e_combo)
            pleb_pairs_match = re.search(pattern=r'^pairs.*_(22|33|44|55|66|77|88|99|1010|JJ|TT)$', string=e_combo)
            monsters_match = re.search(pattern=r'^(trips|straights|flushes|quads|straight_flushes)_', string=e_combo)

            if pleb_pairs_match:
                pleb_pairs_count += 1
            if prem_pairs_match:
                prem_pairs_count += 1
            if monsters_match:
                monsters_count += 1
        else: # Execute after all iterations of the for loop
            if monsters_count + prem_pairs_count + num_double_pairs(n=pleb_pairs_count) >= 3:
                print('has prem pair')
                fantasy_hands_dict.update({key_tuple:tup_of_list_cardtuples})
                found_flag = True
                break
            else:
                pass
    
    else:
        for e_combo in key_tuple:
            # prem_pairs_match = re.search(pattern=r'^(pairs).*(QQ|KK|AA)$', string=e_combo)
            monsters_match = re.search(pattern=r'^(trips|straights|flushes|quads|straight_flushes)_', string=e_combo)

            # if prem_pairs_match:
            #     prem_pairs_match += 1
            if monsters_match:
                monsters_count += 1
        else:
            if monsters_count + prem_pairs_count >= 3:
                print('3 monsters')
                found_flag = True
                fantasy_hands_dict[key_tuple] = tup_of_list_cardtuples
                break
                
            else:
                print('No fantasyland')
            
fantasy_hands_dict

No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland
No fantasyland


{}

In [47]:
def is_fantasyland_true(all_dicts=obj_2['all_dicts']):
    max_cards_used = 0
    max_keys = 0
    fantasy_hands_dict = {}
    count = 1
    for key_tuple, tup_of_list_cardtuples in all_dicts.items():
        prem_pairs_count = 0
        prem_trips_count = 0
        pleb_pairs_count = 0
        regular_trips_count = 0
        monsters_count = 0
        found_flag = False
        found_prem_pair = False

        # If there is a prem_pair or prem_trip....
        has_prem_pair = any(re.search(pattern=r'^(pairs|trips).*(Q{2,3}|K{2,3}|A{2,3})$', string=e_combo) for e_combo in key_tuple)
        
        # .. then we can count pleb_pairs
        has_pleb_pair = any(re.search(pattern=r'^pairs.*_(22|33|44|55|66|77|88|99|1010|JJ|TT)$', string=e_combo) for e_combo in key_tuple)
        
        if has_prem_pair:
            for e_combo in key_tuple:
                prem_pairs_match = re.search(pattern=r'^pairs.*(QQ|KK|AA)$', string=e_combo)
                pleb_pairs_match = re.search(pattern=r'^pairs.*_(22|33|44|55|66|77|88|99|1010|JJ|TT)$', string=e_combo)
                monsters_match = re.search(pattern=r'^(trips|straights|flushes|quads|straight_flushes)_', string=e_combo)
    
                if pleb_pairs_match:
                    pleb_pairs_count += 1
                if prem_pairs_match:
                    prem_pairs_count += 1
                if monsters_match:
                    monsters_count += 1
            else: # Execute after all iterations of the for loop
                if monsters_count + prem_pairs_count + num_double_pairs(n=pleb_pairs_count) >= 3:
                    print('has prem pair')
                    fantasy_hands_dict.update({key_tuple:tup_of_list_cardtuples})
                    found_flag = True
                    break
                else:
                    pass
        
        else: # If no prem_pairs, we can look to monsters...
            for e_combo in key_tuple:
                # prem_pairs_match = re.search(pattern=r'^(pairs).*(QQ|KK|AA)$', string=e_combo)
                monsters_match = re.search(pattern=r'^(trips|straights|flushes|quads|straight_flushes)_', string=e_combo)
    
                # if prem_pairs_match:
                #     prem_pairs_match += 1
                if monsters_match:
                    monsters_count += 1
            else:
                if monsters_count + prem_pairs_count >= 3:
                    print('3 monsters')
                    found_flag = True
                    
                    fantasy_hands_dict[key_tuple] = tup_of_list_cardtuples
                    break
                else:
                    continue
                    print('No fantasyland')

    return fantasy_hands_dict

In [48]:
# Invoke 
is_fantasyland_true(obj_2['all_dicts'])

{}

## Automate the execution of edgecases
> We 

In [49]:
%%time
# Trying to invoke #generate_3_monsters(enable_custom_monsters=True, custom_monsters=['trips', 'straights', 'flushes'])
for key, subdict in my_hands.items():
    obj_1 = DECONSTRUCTOR(is_random=False, is_id=True,
                          edgecase_str=my_hands[key]['Hand'])
    print(f'Remarks\n {subdict["Remarks"]}')
    pprint(obj_1)
    obj_2 = grouper3(obj_1) # grouper(obj_2)
    fantasy_hand = is_fantasyland_true(obj_2['all_dicts'])
    if not fantasy_hand:
        print('NO FANTASY')
        pprint(obj_2['best_hands'])
    else:
        print('FANTASY!!')
        pprint(fantasy_hand)

CASE 3 : FLUSH PRESNET AND STRAIGHT ABSENT
Flush : singletons in Clubs : [('2', 'Diamonds', 'ID_3'), ('4', 'Diamonds', 'ID_4'), ('8', 'Diamonds', 'ID_6')] 
Flush : multipletons in Clubs : [('Q', 'Diamonds', 'ID_2'), ('6', 'Diamonds', 'ID_5')] 
Remarks
 Contains 3 Queens where the the Queen of Diamonds is part of a 5 card flush. No fantasyland
{'plebs': [('2', 'Diamonds', 'ID_3'),
           ('4', 'Diamonds', 'ID_4'),
           ('6', 'Diamonds', 'ID_5'),
           ('8', 'Diamonds', 'ID_6'),
           ('3', 'Clubs', 'ID_10'),
           ('6', 'Hearts', 'ID_11'),
           ('9', 'Spades', 'ID_12'),
           ('J', 'Hearts', 'ID_13')],
 'prems': [('Q', 'Spades', 'ID_1'),
           ('Q', 'Diamonds', 'ID_2'),
           ('Q', 'Hearts', 'ID_7'),
           ('K', 'Spades', 'ID_8'),
           ('A', 'Clubs', 'ID_9')],
 'ranks': {'2': {'length': 1, 'partition': [('2', 'Diamonds', 'ID_3')]},
           '3': {'length': 1, 'partition': [('3', 'Clubs', 'ID_10')]},
           '4': {'length': 1,

In [50]:
list_of_lists = [['a'], ['b1', 'b2'], ['c1', 'c2', 'c3']]
list(product(*list_of_lists))

[('a', 'b1', 'c1'),
 ('a', 'b1', 'c2'),
 ('a', 'b1', 'c3'),
 ('a', 'b2', 'c1'),
 ('a', 'b2', 'c2'),
 ('a', 'b2', 'c3')]

# Monte carlo simulation

In [55]:
%%time
trials = 1000
num_of_trues = 0
for i in range(1, trials+1):
    # Trying to invoke
    print(f'\n Simulation {i}: \n')
    obj_1 = DECONSTRUCTOR(is_random=True, is_id=True, edgecase_str=None)
    print('Sorted sample:\n', obj_1['sorted_sample'], sep='')
    obj_2 = grouper3(obj_1) # grouper(obj_2)
    fantasy_hand = is_fantasyland_true(obj_2['all_dicts'])
    if fantasy_hand:
        pprint(fantasy_hand)
        print('FANTASY!!')
        num_of_trues +=1
    else:
        continue
        print('NO FANTASY')


 Simulation 1: 

CASE 0 : BOTH FLUSH AND STRAIGHT ABSENT
Sorted sample:
[('2', 'Hearts', 'ID_13'), ('2', 'Clubs', 'ID_12'), ('3', 'Clubs', 'ID_8'), ('6', 'Spades', 'ID_3'), ('6', 'Hearts', 'ID_11'), ('6', 'Diamonds', 'ID_6'), ('8', 'Spades', 'ID_5'), ('9', 'Diamonds', 'ID_1'), ('10', 'Spades', 'ID_7'), ('10', 'Clubs', 'ID_2'), ('J', 'Hearts', 'ID_9'), ('A', 'Hearts', 'ID_10'), ('A', 'Clubs', 'ID_4')]
Registered units : 

{'pairs_combo1_22': [('2', 'Clubs', 'ID_12'), ('2', 'Hearts', 'ID_13')],
 'pairs_combo2_66': [('6', 'Spades', 'ID_3'), ('6', 'Diamonds', 'ID_6')],
 'pairs_combo3_66': [('6', 'Spades', 'ID_3'), ('6', 'Hearts', 'ID_11')],
 'pairs_combo4_66': [('6', 'Diamonds', 'ID_6'), ('6', 'Hearts', 'ID_11')],
 'pairs_combo5_1010': [('10', 'Clubs', 'ID_2'), ('10', 'Spades', 'ID_7')],
 'pairs_combo6_AA': [('A', 'Clubs', 'ID_4'), ('A', 'Hearts', 'ID_10')],
 'trips_combo1_666': [('6', 'Spades', 'ID_3'),
                      ('6', 'Diamonds', 'ID_6'),
                      ('6', 'Hearts'

In [53]:
num_of_trues

301

In [ ]:
i

## Appendix

In [ ]:
obj_1 = DECONSTRUCTOR(is_random=False, is_id=True,
                      edgecase_str=my_hands['06_2quads and a straight flush'])

pprint(obj_1)
obj_2 = grouper3(obj_1) # grouper(obj_2)
fantasy_hand = is_fantasyland_true(obj_2['all_dicts'])
if not fantasy_hand:
    print('NO FANTASY')
    pprint(obj_2['best_hands'])
else:
    print('FANTASY!!')
    pprint(fantasy_hand)

In [ ]:
remove_subsets([['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'],
                ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']])

In [ ]:
li2 = [['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'],
                ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']]
list(chain(*li2))

In [ ]:
pprint(my_hands)

In [ ]:
obj_1['sy_repo']

In [ ]:
limited_flushes = {f'flushes_combo{i}_{value[-1][0]}': value for i, value in enumerate([], start=1)}

## Printing a random configuration with the best synergy

In [ ]:
# pprint(obj_3['best_hands'])
import random
hand_num, hands_part = random.choice(list(obj_3['best_hands'].items()))
# Extract the only pair
(keys_tuple, values_tuple) = next(iter(hands_part.items())) #hands_part.items() makes a tuple of ((keys), (values))
# Zip into a new dict
reoriented = dict(zip(keys_tuple, values_tuple))

# View our sample 
for card in sorted(obj_1['sample'], key=lambda card : (ranks.index(card[0]), card[1])):
    print(card)
pprint(reoriented)

## Returning the best configuration with the most numerous keys

In [ ]:
# Find the top-level key whose sub-dict contains the longest tuple key
max_entry = max(
    obj_3['best_hands'].items(),
    key=lambda combos_and_subdict: max(len(k) for k in combos_and_subdict[1].keys())
)
# Reconstruct a filtered dict containing only that entry
filtered_dict = {max_entry[0]: max_entry[1]}

# Extract the only pair - max_entry
(keys_tuple, values_tuple) = next(iter(max_entry[1].items())) #hands_part.items() makes a tuple of ((keys), (values))
# Zip into a new dict
best_synergy = dict(zip(keys_tuple, values_tuple))

# View our sample 
for card in sorted(obj_1['sample'], key=lambda card : (ranks.index(card[0]), card[1])):
    print(card)
best_synergy

In [ ]:
obj_3['best_hands']

## Adding any unsynergised cards

In [ ]:
# Get the synergised cardd
synergised_cards = list(itertools.chain(*best_synergy.values()))
my_thirteen = obj_1['sample'].copy()
# Get the unsynergised cards
for e_card in synergised_cards:
    my_thirteen.pop(my_thirteen.index(e_card))
# Append any synergised cards
if my_thirteen:
    best_synergy.setdefault('unsynergy', []).extend(my_thirteen)
best_synergy

In [ ]:
import re

def num_double_pairs_defunct(n):
    """
    Num_of_pairs -> num_of_double_pairs
    1 -> 0
    2 -> 1
    3 -> 1
    4 -> 2
    5 -> 2
    6 -> 3
    """
    return (n + 1) // 2 - (1 if n == 1 else 0)

def num_double_pairs(n):
    if n in (0, 1):
        return 0
    elif n in (2, 3):
        return 1
    elif n in (4, 5):
        return 2
    elif n in (6, 7):
        return 3
    elif n in (8, 9):
        return 4
    elif n in (10, 11):
        return 5
    else:
        raise ValueError("n out of expected range")

def is_fantasyland(best_synergy_hand=best_synergy):
    """
    """
    prem_pairs_count = 0
    prem_trips_count = 0
    regular_pairs_count = 0
    regular_trips_count = 0
    monsters_count = 0
    for e_combo in best_synergy.keys():
        pairs_match = re.search(pattern=r'^(pairs).*(\w{2,4})$', string=e_combo)
        trips_match = re.search(pattern=r'^(trips).*(\w{3,6})$', string=e_combo)
        monsters_match = re.search(pattern=r'^(straights|flushes|straight_flushes)_', string=e_combo)
        if pairs_match:
            pairs_prefix = pairs_match.group(2)
            if pairs_prefix in ['QQ', 'KK', 'AA']:
                prem_pairs_count += 1
                print(f"here is your prem_pair {pairs_prefix}")
            else:
                regular_pairs_count += 1
                print(f"here is your pleb_pair {pairs_prefix}")
        if trips_match:
            trips_prefix = trips_match.group(2)
            if trips_prefix in ['QQQ', 'KKK', 'AAA']:
                prem_trips_count += 1
                print(f"here is your prem_trips {trips_prefix}")
            else:
                regular_trips_count += 1
                print(f"here is your pleb_trips {trips_prefix}")
        if monsters_match:
            captured_monster = monsters_match.group(1)
            monsters_count += 1
            # print(f'Here is the monster : {captured_monster}')
    
    # These are hands that can only fit to the TOP
    basic_requirement = prem_pairs_count + prem_trips_count + regular_trips_count
        
    points_to_fantasy = num_double_pairs(n=regular_pairs_count) + prem_pairs_count + prem_trips_count + regular_trips_count + monsters_count
    print(f'The basic requirment is {basic_requirement}')
    print(f'The points to fantasyland is {points_to_fantasy}')
    if points_to_fantasy >= 3 and basic_requirement >= 1:
        print('Case 1')
        return True
    elif monsters_count + regular_trips_count + prem_trips_count >= 3:
        print('Case 2')
        return True
    else:
        return False

In [ ]:
prem_pairs_count = 0
prem_trips_count = 0
regular_pairs_count = 0
regular_trips_count = 0
monsters_count = 0
for e_combo in best_synergy.keys():
    pairs_match = re.search(pattern=r'^(pairs).*(\w{2,4})$', string=e_combo)
    trips_match = re.search(pattern=r'^(trips).*(\w{3,6})$', string=e_combo)
    monsters_match = re.search(pattern=r'^(straights|flushes|straight_flushes)_', string=e_combo)
    if pairs_match:
        pairs_prefix = pairs_match.group(2)
        if pairs_prefix in ['QQ', 'KK', 'AA']:
            prem_pairs_count += 1
            print(f"here is your prem_pair {pairs_prefix}")
        else:
            regular_pairs_count += 1
            print(f"here is your pleb_pair {pairs_prefix}")
    if trips_match:
        trips_prefix = trips_match.group(2)
        if trips_prefix in ['QQQ', 'KKK', 'AAA']:
            prem_trips_count += 1
            print(f"here is your prem_trips {trips_prefix}")
        else:
            regular_trips_count += 1
            print(f"here is your pleb_trips {trips_prefix}")
    if monsters_match:
        captured_monster = monsters_match.group(1)
        monsters_count += 1
        # print(f'Here is the monster : {captured_monster}')

# These are hands that can only fit to the TOP
basic_requirement = prem_pairs_count + prem_trips_count + regular_trips_count
    
points_to_fantasy = num_double_pairs(n=regular_pairs_count) + prem_pairs_count + prem_trips_count + regular_trips_count + monsters_count
print(f'The basic requirment is {basic_requirement}')
print(f'The points to fantasyland is {points_to_fantasy}')
# if points_to_fantasy >= 3 and basic_requirement >= 1:
#     print('Case 1')
#     return True
# elif monsters_count + regular_trips_count + prem_trips_count >= 3:
#     print('Case 2')
#     return True
# else:
#     return False

In [ ]:
best_synergy

In [ ]:
is_fantasyland(best_synergy_hand=best_synergy)

In [ ]:
%%time
trials = 1000
num_of_trues = 0
for i in range(1, trials+1):
    # Trying to invoke
    obj_1 = DECONSTRUCTOR(is_random=True, is_id=True, edgecase_str=None)
    
    pprint(obj_1['suits'])
    obj_2 = PLEB_DECONSTRUCTOR()
    pprint(obj_2)  
    obj_3 = grouper3(obj_1, obj_2) # grouper(obj_2)
    # pprint(obj_3['best_hands'])
    # Find the top-level key whose sub-dict contains the longest tuple key
    max_entry = max(
        obj_3['best_hands'].items(),
        key=lambda combos_and_subdict: max(len(k) for k in combos_and_subdict[1].keys())
    )
    # Reconstruct a filtered dict containing only that entry
    filtered_dict = {max_entry[0]: max_entry[1]}
    
    # Extract the only pair - max_entry
    (keys_tuple, values_tuple) = next(iter(max_entry[1].items())) #hands_part.items() makes a tuple of ((keys), (values))
    # Zip into a new dict
    best_synergy = dict(zip(keys_tuple, values_tuple))
    
    # View our sample 
    for card in sorted(obj_1['sample'], key=lambda card : (ranks.index(card[0]), card[1])):
        print(card)
    
    # Get the synergised cardd
    synergised_cards = list(itertools.chain(*best_synergy.values()))
    my_thirteen = obj_1['sample'].copy()
    # Get the unsynergised cards
    for e_card in synergised_cards:
        my_thirteen.pop(my_thirteen.index(e_card))
    # Append any synergised cards
    if my_thirteen:
        best_synergy.setdefault('unsynergy', []).extend(my_thirteen)
    
    # View the best_synergy
    pprint(best_synergy)
    
    result = is_fantasyland(best_synergy_hand=best_synergy)
    if result:
        num_of_trues += 1

print(num_of_trues)

In [ ]:
num_of_trues